<a href="https://colab.research.google.com/github/ArthurChan-1111/Bitcoin_prediction/blob/main/Master_dataset_Fear%20and%20greed_by_year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Packages

In [1]:
# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install pandas_ta
    !pip install numpy==1.23.5
    !pip install pandas
    !pip install seaborn
    !pip install xgboost

    !git clone https://github.com/ArthurChan-1111/Bitcoin_prediction.git

    %cd Bitcoin_prediction
    %pip install pandas_ta
else:
    print("Not running in Colab, skipping git clone and directory change")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=68dbaadb18d36507a78f118a62540ae06ad5097ade3594249723893216217aad
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 64.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatibl

Cloning into 'Bitcoin_prediction'...
remote: Enumerating objects: 595, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 595 (delta 49), reused 15 (delta 15), pack-reused 535 (from 2)
Receiving objects: 100% (595/595), 215.58 MiB | 21.04 MiB/s, done.
Resolving deltas: 100% (322/322), done.
/content/Bitcoin_prediction


In [2]:
# General Libraries
import numpy as np
import pandas as pd
import pandas_ta as ta
import csv
import warnings
from tqdm import tqdm
from tqdm.auto import tqdm
from scipy import stats
from scipy.stats import zscore

# Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt
import pylab as py

# Data Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# General Model Selection and Metrics
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV,
    TimeSeriesSplit,
    RepeatedKFold,
    cross_val_score,
    StratifiedKFold
)

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score
)

# Regression Models and Tools
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    RidgeCV,
    LassoCV,
    ElasticNetCV
)

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    BaggingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    StackingRegressor
)
from xgboost import XGBRegressor

# Stats Models
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import (
    variance_inflation_factor,
    summary_table,
    OLSInfluence
)
from statsmodels.regression.linear_model import OLSResults
from statsmodels.stats.stattools import durbin_watson as dwtest
from statsmodels.sandbox.stats.runs import runstest_1samp
from statsmodels.stats.diagnostic import het_white
from statsmodels.compat import lzip

# Classification Models and Metrics
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    roc_auc_score
)

# Settings
warnings.filterwarnings("ignore")
pd.set_option("display.float_format", lambda x: "%.4f" % x)
pd.set_option("display.max_columns", None)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

#Load dataset

In [ ]:
btc_data = pd.read_csv('Bitcoin Historical Data.csv', on_bad_lines='skip', lineterminator='\n')
# Check if 'Change %\r' exists and rename it to 'Change'
if 'Change %\r' in btc_data.columns:
    btc_data.rename(columns={'Change %\r': 'Change'}, inplace=True)
btc_data.tail(5)

In [ ]:
# Convert the dictionary to a DataFrame
btc_data = pd.DataFrame(btc_data)

# Parse the "Date" column into datetime format
btc_data["Date"] = pd.to_datetime(btc_data["Date"], format="%m/%d/%Y")

# Sort the data by date in ascending order
btc_data.sort_values(by="Date", ascending=True, inplace=True)


In [ ]:
gold_price_data = pd.read_csv('gold_price_data.csv', on_bad_lines='skip', lineterminator='\n', sep=';')
# Check if 'Gold_Volume %\r' exists and rename it to 'Gold_Volume %'
if 'Gold_Volume\r' in gold_price_data.columns:
    gold_price_data.rename(columns={'Gold_Volume\r': 'Gold_Volume'}, inplace=True)
gold_price_data.tail(5)

In [ ]:
risk_free_data = pd.read_csv('risk_free_data.csv', on_bad_lines='skip', lineterminator='\n', sep=',')
# rename the Yield_Spread\r to Yield_Spread
risk_free_data.rename(columns={'Yield_Spread\r': 'Yield_Spread'}, inplace=True)
risk_free_data.tail(5)

In [ ]:
spy_data = pd.read_csv('spy_data.csv', on_bad_lines='skip', lineterminator='\n', sep=',')
# Check if 'SPY_Volume\r' exists and rename it to 'SPY_Volume'
if 'SPY_Volume\r' in spy_data.columns:
    spy_data.rename(columns={'SPY_Volume\r': 'SPY_Volume'}, inplace=True)
spy_data.tail(5)

In [ ]:
eth_data = pd.read_csv('eth_data.csv', on_bad_lines='skip', lineterminator='\n', sep=',')

# Remove the dollar sign and convert 'ETH_Price' to a numeric type
eth_data['ETH_Price'] = eth_data['ETH_Price'].replace('[\$,]', '', regex=True).astype(float)

# Check if 'ETH_Volume %\r' exists and rename it to 'ETH_Volume %'
if 'ETH_Volume\r' in eth_data.columns:
    eth_data.rename(columns={'ETH_Volume\r': 'ETH_Volume'}, inplace=True)

eth_data.tail(5)

In [ ]:
#Change the 'Date' to datetime format
gold_price_data['Date'] = pd.to_datetime(gold_price_data['Date'])
risk_free_data['Date'] = pd.to_datetime(risk_free_data['Date'])
spy_data['Date'] = pd.to_datetime(spy_data['Date'])
eth_data['Date'] = pd.to_datetime(eth_data['Date'])

#Data Preprocessing

In [ ]:
#Function to clean and convert volume data
def clean_volume(volume):
    if isinstance(volume, str):  # Check if the value is a string
        volume = volume.replace(',', '')  # Remove commas
        if 'B' in volume:  # If the value contains 'B' (billions)
            return float(volume.replace('B', '')) * 1_000_000_000
        elif 'M' in volume:  # If the value contains 'M' (millions)
            return float(volume.replace('M', '')) * 1_000_000
        elif 'K' in volume:  # If the value contains 'K' (thousands)
            return float(volume.replace('K', '')) * 1_000
        else:  # If no suffix is present, convert to float directly
            return float(volume)
    return np.nan  # Handle unexpected cases

# Apply the cleaning function to the volume column
btc_data["Volume"] = btc_data["Vol. ('000)"].apply(clean_volume)
btc_data = btc_data.drop("Vol. ('000)", axis=1)

# 4. Remove '%' from "Change %" and convert to numeric
btc_data.rename(columns={"Change %\r": "Change"}, inplace=True)
btc_data["Change"] = btc_data["Change"].str.replace("%", "").str.strip().astype(float)

In [ ]:
# Define the missing data as a dictionary
missing_data = {
    'Date': '01/12/2025',
    'Price': 94541.8,
    'Open': 94600.0,
    'High': 95384.3,
    'Low': 93711.2,
    'Volume': 17.60,  # Assuming 'Vol. (\'000)' is in thousands
    'Change': -0.07  # Assuming 'Change %' is already converted to a float
}

# Convert the dictionary to a DataFrame
missing_data_df = pd.DataFrame([missing_data])

# Ensure the 'Date' column is in datetime format
missing_data_df['Date'] = pd.to_datetime(missing_data_df['Date'])

# Append the missing data to btc_data
btc_data = pd.concat([btc_data, missing_data_df], ignore_index=True)

# Filter the data for the date range 01/10/2025 to 01/20/2025
filtered_data = btc_data[(btc_data['Date'] >= '2025-01-10') & (btc_data['Date'] <= '2025-01-20')]

# Display the filtered data
print(filtered_data)

In [ ]:
# Day of the week (0=Monday, 6=Sunday)
btc_data["Day_of_Week"] = btc_data["Date"].dt.dayofweek

# Week of the year
btc_data["Week_of_Year"] = btc_data["Date"].dt.isocalendar().week

# Month of the year
btc_data["Month"] = btc_data["Date"].dt.month

# Quarter of the year
btc_data["Quarter"] = btc_data["Date"].dt.quarter

# Year
btc_data["Year"] = btc_data["Date"].dt.year

## Signal Detection

In [ ]:
# Calculate RSI, EMA, SMA, and MACD
btc_data["RSI_6"] = ta.rsi(btc_data["Price"], length=6)  # Relative Strength Index
btc_data["RSI_12"] = ta.rsi(btc_data["Price"], length=12)  # Relative Strength Index
btc_data["EMA_14"] = ta.ema(btc_data["Price"], length=14)  # Exponential Moving Average
btc_data["SMA_14"] = ta.sma(btc_data["Price"], length=14)  # Simple Moving Average

# Add On-Balance Volume (OBV)
btc_data["OBV"] = ta.obv(btc_data["Price"], btc_data["Volume"])

# Calculate MACD --------------------------------------------------------------------------------------
macd = ta.macd(btc_data["Price"], fast=12, slow=26, signal=9)
btc_data["MACD"] = macd["MACD_12_26_9"]
btc_data["MACD_Signal"] = macd["MACDs_12_26_9"]

# Create MACD_buy: 1 if MACD crosses above MACD_Signal, otherwise 0
btc_data["MACD_buy"] = 0  # Default to 0 (no buy signal)
btc_data.loc[
    (btc_data["MACD"] > btc_data["MACD_Signal"]) &  # MACD is above Signal
    (btc_data["MACD"].shift(1) <= btc_data["MACD_Signal"].shift(1)),  # Previous MACD was below or equal to Signal
    "MACD_buy"] = 1  # Buy signal

# Create MACD_sell: 1 if MACD crosses below MACD_Signal, otherwise 0
btc_data["MACD_sell"] = 0  # Default to 0 (no sell signal)
btc_data.loc[
    (btc_data["MACD"] <= btc_data["MACD_Signal"]) &  # MACD is below or equal to Signal
    (btc_data["MACD"].shift(1) > btc_data["MACD_Signal"].shift(1)),  # Previous MACD was above Signal
    "MACD_sell"] = 1  # Sell signal

# New Variable: MACD_above_signal (1 if MACD > MACD_Signal, otherwise 0)
btc_data["MACD_above_signal"] = (btc_data["MACD"] > btc_data["MACD_Signal"]).astype(int)

# New Variable: MACD_below_signal (1 if MACD <= MACD_Signal, otherwise 0)
btc_data["MACD_below_signal"] = (btc_data["MACD"] <= btc_data["MACD_Signal"]).astype(int)

# Create MACD_cum_buy: Accumulate consecutive days where MACD_above_signal is 1
btc_data["MACD_cum_buy"] = (
    btc_data["MACD_above_signal"]
    .groupby((btc_data["MACD_above_signal"] == 0).cumsum())  # Group by resets when MACD_above_signal is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["MACD_above_signal"] == 0, "MACD_cum_buy"] = 0  # Reset to 0 where the condition is not met

# Create MACD_cum_sell: Accumulate consecutive days where MACD_below_signal is 1
btc_data["MACD_cum_sell"] = (
    btc_data["MACD_below_signal"]
    .groupby((btc_data["MACD_below_signal"] == 0).cumsum())  # Group by resets when MACD_below_signal is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["MACD_below_signal"] == 0, "MACD_cum_sell"] = 0  # Reset to 0


# Calculate Bollinger Bands --------------------------------------------------------------------------------------
bb = ta.bbands(btc_data["Price"], length=20, std=2)  # 20-period BB with 2 standard deviations
btc_data["BB_upper"] = bb.get("BBU_20_2.0")  # Upper Bollinger Band
btc_data["BB_lower"] = bb.get("BBL_20_2.0")  # Lower Bollinger Band

# Create BB_Buy: 1 if Price crosses below the Lower Band, otherwise 0
btc_data["BB_Buy"] = 0  # Default to 0 (no buy signal)
btc_data.loc[
    (btc_data["Price"] < btc_data["BB_lower"]) &  # Price is below the Lower Band
    (btc_data["Price"].shift(1) >= btc_data["BB_lower"].shift(1)),  # Previous Price was above or equal to Lower Band
    "BB_Buy"] = 1  # Buy signal

# Create BB_Sell: 1 if Price crosses above the Upper Band, otherwise 0
btc_data["BB_Sell"] = 0  # Default to 0 (no sell signal)
btc_data.loc[
    (btc_data["Price"] > btc_data["BB_upper"]) &  # Price is above the Upper Band
    (btc_data["Price"].shift(1) <= btc_data["BB_upper"].shift(1)),  # Previous Price was below or equal to Upper Band
    "BB_Sell"] = 1  # Sell signal

# New Variable: Price_above_lower_band (1 if Price > BB_lower, otherwise 0)
btc_data["Price_below_BB_lower"] = (btc_data["Price"] < btc_data["BB_lower"]).astype(int)

# New Variable: Price_below_upper_band (1 if Price < BB_upper, otherwise 0)
btc_data["Price_above_BB_upper"] = (btc_data["Price"] > btc_data["BB_upper"]).astype(int)

# Create BB_cum_Buy: Accumulate consecutive days where Price_below_BB_lower is 1
btc_data["BB_cum_Buy"] = (
    btc_data["Price_below_BB_lower"]
    .groupby((btc_data["Price_below_BB_lower"] == 0).cumsum())  # Group by resets when Price_below_BB_lower is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["Price_below_BB_lower"] == 0, "BB_cum_Buy"] = 0  # Reset to 0 where the condition is not met

# Create BB_cum_Sell: Accumulate consecutive days where Price_above_BB_upper is 1
btc_data["BB_cum_Sell"] = (
    btc_data["Price_above_BB_upper"]
    .groupby((btc_data["Price_above_BB_upper"] == 0).cumsum())  # Group by resets when Price_above_BB_upper is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["Price_above_BB_upper"] == 0, "BB_cum_Sell"] = 0  # Reset to 0 where the condition is not met

# Calculate ATR --------------------------------------------------------------------------------------
btc_data["ATR"] = ta.atr(btc_data["High"], btc_data["Low"], btc_data["Price"], length=14)

# Set Stop-Loss Levels (Example with Long Trade)
atr_multiplier = 2
btc_data["Stop_Loss_Long"] = btc_data["Price"] - (btc_data["ATR"] * atr_multiplier) #Stop-Loss for Buy (Long) Trade
btc_data["Stop_Loss_Short"] = btc_data["Price"] + (btc_data["ATR"] * atr_multiplier) #Stop-Loss for Sell (Short) Trade

# Calculate VWAP --------------------------------------------------------------------------------------
btc_data.set_index("Date", inplace=True) # Set the "Date" column as the index
btc_data["VWAP"] = ta.vwap(btc_data["High"], btc_data["Low"], btc_data["Price"], btc_data["Volume"])

# Create VWAP_Buy: 1 if Price crosses above VWAP, otherwise 0
btc_data["VWAP_Buy"] = 0  # Default to 0 (no buy signal)
btc_data.loc[
    (btc_data["Price"] > btc_data["VWAP"]) &  # Price is above VWAP
    (btc_data["Price"].shift(1) <= btc_data["VWAP"].shift(1)),  # Previous Price was below or equal to VWAP
    "VWAP_Buy"] = 1  # Buy signal

# Create VWAP_Sell: 1 if Price crosses below VWAP, otherwise 0
btc_data["VWAP_Sell"] = 0  # Default to 0 (no sell signal)
btc_data.loc[
    (btc_data["Price"] < btc_data["VWAP"]) &  # Price is below VWAP
    (btc_data["Price"].shift(1) >= btc_data["VWAP"].shift(1)),  # Previous Price was above or equal to VWAP
    "VWAP_Sell"] = 1  # Sell signal

# Fear and greed index
fear_and_greed_index = pd.read_csv('fear_and_greed_index.csv')
# rename the date column in fear_and_greed_index to Date
fear_and_greed_index.rename(columns={"date": "Date", "value": "Fear_and_Greed_Index"}, inplace=True)
# convert the date column to datetime
fear_and_greed_index["Date"] = pd.to_datetime(fear_and_greed_index["Date"], format="%Y-%m-%d")
# dropping the timestamp, value_classification, time_until_update columns
fear_and_greed_index.drop(columns=["timestamp", "value_classification", "time_until_update"], inplace=True)
fear_and_greed_index.head()

# Join the two dataframes on the "Date" column, dropping
btc_data = btc_data.merge(fear_and_greed_index, on="Date", how="left")
btc_data.head(20)

# Support and Resistance Levels
btc_data["Support"] = btc_data["Low"].rolling(window=20).min()  # Lowest low in the past 20 days
btc_data["Resistance"] = btc_data["High"].rolling(window=20).max()  # Highest high in the past 20 days

# Add two columns for extreme fear and extreme greed
btc_data["Extreme_Fear"] = np.where(btc_data["Fear_and_Greed_Index"] < 20, 1, 0)
btc_data["Extreme_Greed"] = np.where(btc_data["Fear_and_Greed_Index"] > 80, 1, 0)
btc_data.describe().T

# Use 70 in RSI to classify overbought and 30 to classify oversold
btc_data["RSI_12_Oversold"] = np.where(btc_data["RSI_12"] < 30, 1, 0)  # Create RSI_12_Oversold column
btc_data.loc[btc_data["RSI_12_Oversold"] == 0, "Consecutive_RSI_12_Oversold"] = 0  # Reset to 0 where the condition is not met
btc_data["RSI_12_Overbought"] = np.where(btc_data["RSI_12"] > 70, 1, 0)

# Adding RSI_6 greater than RSI_12 to show bullish divergence and conversely
btc_data["RSI_Divergence"] = np.where(btc_data["RSI_6"] > btc_data["RSI_12"], 1, 0)
btc_data.describe().T


# Count consecutive appearances of Extreme_Fear
btc_data["Consecutive_Extreme_Fear"] = (
    btc_data["Extreme_Fear"]
    .groupby((btc_data["Extreme_Fear"] == 0).cumsum())  # Group by resets when Extreme_Fear is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["Extreme_Fear"] == 0, "Consecutive_Extreme_Fear"] = 0  # Reset to 0 where the condition is not met

# Count consecutive appearances of Extreme_Greed
btc_data["Consecutive_Extreme_Greed"] = (
    btc_data["Extreme_Greed"]
    .groupby((btc_data["Extreme_Greed"] == 0).cumsum())  # Group by resets when Extreme_Greed is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["Extreme_Greed"] == 0, "Consecutive_Extreme_Greed"] = 0  # Reset to 0 where the condition is not met

# Forward fill the missing value in 'Fear_and_Greed_Index'
btc_data['Fear_and_Greed_Index'] = btc_data['Fear_and_Greed_Index'].fillna(method='ffill')

# Count consecutive appearances of RSI_12_Overbought
btc_data["Consecutive_RSI_12_Overbought"] = (
    btc_data["RSI_12_Overbought"]
    .groupby((btc_data["RSI_12_Overbought"] == 0).cumsum())  # Group by resets when RSI_12_Overbought is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["RSI_12_Overbought"] == 0, "Consecutive_RSI_12_Overbought"] = 0  # Reset to 0 where the condition is not met

# Count consecutive appearances of RSI_12_Oversold
btc_data["Consecutive_RSI_12_Oversold"] = (
    btc_data["RSI_12_Oversold"]
    .groupby((btc_data["RSI_12_Oversold"] == 0).cumsum())  # Group by resets when RSI_12_Oversold is 0
    .cumcount()  # Count consecutive days starting from 1
)

In [ ]:
#Add continous variables

#ATR ratio
btc_data['ATR_Ratio'] = (btc_data['ATR'] / btc_data['Price']) * 100

# Calculate 50-day and 200-day moving averages
btc_data['MA50'] = btc_data['Price'].rolling(window=50).mean()
btc_data['MA200'] = btc_data['Price'].rolling(window=200).mean()

# Calculate Price relative to 50-day MA (%)
btc_data['Price_vs_MA50'] = ((btc_data['Price'] - btc_data['MA50']) / btc_data['MA50']) * 100

# Calculate Price relative to 200-day MA (%)
btc_data['Price_vs_MA200'] = ((btc_data['Price'] - btc_data['MA200']) / btc_data['MA200']) * 100

# Calculate MA Distance Ratio (Ratio of MA50 to MA200 distance)
btc_data['MA_Distance_Ratio'] = ((btc_data['MA50'] - btc_data['MA200']) / btc_data['Price']) * 100

In [ ]:
from scipy.stats import linregress

# Ensure btc_data is sorted by Date
btc_data = btc_data.sort_values('Date')

# 1. Volume_Ratio: Current volume / 20-day avg volume
btc_data['Avg_Volume_20'] = btc_data['Volume'].rolling(window=20).mean()
btc_data['Volume_Ratio'] = btc_data['Volume'] / btc_data['Avg_Volume_20']

# 2. OBV_Slope: 5-day slope of On-Balance Volume (OBV)
# Calculate OBV
btc_data['OBV'] = (np.sign(btc_data['Price'].diff()) * btc_data['Volume']).fillna(0).cumsum()

# Calculate 5-day slope of OBV
def calculate_slope(series):
    x = np.arange(len(series))
    slope, _, _, _, _ = linregress(x, series)
    return slope

btc_data['OBV_Slope'] = btc_data['OBV'].rolling(window=5).apply(calculate_slope, raw=False)

In [ ]:
# Calculate the percentage price change over the next 7-day and 2-day (Target Variable for Regression)
btc_data["Pct_Change_7D"] = ((btc_data["Price"].shift(-7) - btc_data["Price"]) / btc_data["Price"]) * 100
btc_data["Pct_Change_2D"] = ((btc_data["Price"].shift(-2) - btc_data["Price"]) / btc_data["Price"]) * 100

# Create binary variables for 7-day and 2-day percentage changes
btc_data["Positive_7D"] = (btc_data["Pct_Change_7D"] > 0).astype(int)  # 1 if positive, 0 if negative
btc_data["Positive_2D"] = (btc_data["Pct_Change_2D"] > 0).astype(int)  # 1 if positive, 0 if negative

## Pattern Detection

In [ ]:
# Asending Triangle
# Sort data by Date
btc_data = btc_data.sort_values('Date')

# Calculate exponential moving average for Resistance_Level
btc_data['Resistance_Level'] = btc_data['High'].ewm(span=7, adjust=False).mean()

# Identify rising trendline (higher lows)
btc_data['Low_Shifted'] = btc_data['Low'].shift(1)
btc_data['Higher_Low'] = (btc_data['Low'] > btc_data['Low_Shifted']).astype(int)

# Detect breakout using rolling mean of Volume
btc_data['Breakout'] = btc_data['Price'] > btc_data['Resistance_Level']

# Add ascending triangle detection signal
btc_data['Ascending_Triangle_Breakout'] = btc_data['Breakout']

# Ensure Breakout is binary (1 or 0)
btc_data['Breakout'] = btc_data['Breakout'].astype(int)

# Ensure Ascending_Triangle_Breakout is binary (1 or 0)
btc_data['Ascending_Triangle_Breakout'] = btc_data['Ascending_Triangle_Breakout'].astype(int)

# Calculate the rolling mean of Volume over 7 days
btc_data['Avg_Volume'] = btc_data['Volume'].rolling(window=7).mean()

# Detect breakout with volume confirmation
btc_data['Breakout_With_Volume'] = ((btc_data['High'] > btc_data['Resistance_Level']) &
                                    (btc_data['Volume'] > btc_data['Avg_Volume'])).astype(int)

# Add ascending triangle detection signal with volume
btc_data['Ascending_Triangle_Breakout_With_Volume'] = btc_data['Breakout_With_Volume']

In [ ]:
# Count the number of breakouts
breakout_count = btc_data['Ascending_Triangle_Breakout_With_Volume'].sum()
print(f"Number of breakouts: {breakout_count}")

In [ ]:
btc_data['Triangle_Completion'] = btc_data['Higher_Low'].rolling(window=7).mean()  # Simulate completion as a rolling mean of Higher_Low
btc_data['Triangle_Height'] = ((btc_data['High'] - btc_data['Low']) / btc_data['Price']).rolling(window=7).mean()  # Height as % of price
btc_data['Triangle_Duration'] = btc_data['Higher_Low'].rolling(window=7).sum()  # Duration as the sum of Higher_Low over 7 days
btc_data['Breakout_Strength'] = ((btc_data['High'] - btc_data['Resistance_Level']) * btc_data['Volume']).rolling(window=7).mean()  # Volume-adjusted breakout strength

In [ ]:
#ADX Pattern (Show the strength of the trend, please only use ADX_Buy_Signal)
!pip install ta
import ta
# Calculate ADX, +DI, and -DI using the ta library
btc_data['ADX'] = ta.trend.adx(btc_data['High'], btc_data['Low'], btc_data['Price'], window=14)
btc_data['Positive_DI'] = ta.trend.adx_pos(btc_data['High'], btc_data['Low'], btc_data['Price'], window=14)
btc_data['Negative_DI'] = ta.trend.adx_neg(btc_data['High'], btc_data['Low'], btc_data['Price'], window=14)

# Define the ADX threshold for a strong trend
adx_threshold = 25

# Generate trade signals
btc_data['ADX_Buy_Signal'] = ((btc_data['ADX'] > adx_threshold) & (btc_data['Positive_DI'] > btc_data['Negative_DI'])).astype(int)
btc_data['ADX_Sell_Signal'] = ((btc_data['ADX'] > adx_threshold) & (btc_data['Negative_DI'] > btc_data['Positive_DI'])).astype(int)

In [ ]:
# Create columns for consecutive ADX Buy and Sell signals
btc_data['Consecutive_ADX_Buy'] = (
    btc_data['ADX_Buy_Signal']
    .groupby((btc_data['ADX_Buy_Signal'] == 0).cumsum())
    .cumcount()
)
btc_data.loc[btc_data['ADX_Buy_Signal'] == 0, 'Consecutive_ADX_Buy'] = 0

btc_data['Consecutive_ADX_Sell'] = (
    btc_data['ADX_Sell_Signal']
    .groupby((btc_data['ADX_Sell_Signal'] == 0).cumsum())
    .cumcount()
)
btc_data.loc[btc_data['ADX_Sell_Signal'] == 0, 'Consecutive_ADX_Sell'] = 0

In [ ]:
# Count the number of breakout signals
breakout_count = btc_data['ADX_Buy_Signal'].sum()
print(f"Number of breakout signals: {breakout_count}")

In [ ]:
# Add continuous features
# ADX_Strength: How far ADX is above the threshold (ADX - 25, 0 if below)
btc_data['ADX_Strength'] = (btc_data['ADX'] - adx_threshold).clip(lower=0)

# ADX_Trend_Duration: Number of consecutive days ADX > 25
btc_data['ADX_Trend_Duration'] = (btc_data['ADX'] > adx_threshold).astype(int).groupby((btc_data['ADX'] <= adx_threshold).astype(int).cumsum()).cumsum()

# ADX_Trend_Direction: +DI minus -DI (positive for uptrend, negative for downtrend)
btc_data['ADX_Trend_Direction'] = btc_data['Positive_DI'] - btc_data['Negative_DI']

In [ ]:
# Calculate Stochastic Oscillator (%K and %D)
btc_data['Stochastic_perK'] = ta.momentum.stoch(
    btc_data['High'], btc_data['Low'], btc_data['Price'], window=14, smooth_window=3
)
btc_data['Stochastic_perD'] = ta.momentum.stoch_signal(
    btc_data['High'], btc_data['Low'], btc_data['Price'], window=14, smooth_window=3
)

# Define Stochastic Buy and Sell Signals
btc_data['Stochastic_Buy_Signal'] = np.where(
    (btc_data['Stochastic_perK'] < 20) & (btc_data['Stochastic_perK'] > btc_data['Stochastic_perD']),
    1,  # Buy Signal
    0   # No Signal
)

btc_data['Stochastic_Sell_Signal'] = np.where(
    (btc_data['Stochastic_perK'] > 80) & (btc_data['Stochastic_perK'] < btc_data['Stochastic_perD']),
    1,  # Sell Signal
    0   # No Signal
)

In [ ]:
# Add consecutive features for Stochastic signals
btc_data['Consecutive_Stochastic_Buy'] = (
    btc_data['Stochastic_Buy_Signal']
    .groupby((btc_data['Stochastic_Buy_Signal'] == 0).cumsum())
    .cumcount()
)

btc_data['Consecutive_Stochastic_Sell'] = (
    btc_data['Stochastic_Sell_Signal']
    .groupby((btc_data['Stochastic_Sell_Signal'] == 0).cumsum())
    .cumcount()
)

In [ ]:
#Bullish and Bearish Engulfing
def detect_bullish_engulfing(data):
    return (
        (data['Price'] > data['Open']) &  # Current candle is bullish
        (data['Price'].shift(1) < data['Open'].shift(1)) &  # Previous candle is bearish
        (data['Open'] < data['Price'].shift(1)) &  # Current open is below previous close
        (data['Price'] > data['Open'].shift(1))    # Current close is above previous open
    )

def detect_bearish_engulfing(data):
    return (
        (data['Price'] < data['Open']) &  # Current candle is bearish
        (data['Price'].shift(1) > data['Open'].shift(1)) &  # Previous candle is bullish
        (data['Open'] > data['Price'].shift(1)) &  # Current open is above previous close
        (data['Price'] < data['Open'].shift(1))    # Current close is below previous open
    )

# Apply the detection functions to btc_data
btc_data['Bullish_Engulfing'] = detect_bullish_engulfing(btc_data).astype(int)
btc_data['Bearish_Engulfing'] = detect_bearish_engulfing(btc_data).astype(int)

# Display the rows where patterns are detected
bullish_patterns = btc_data[btc_data['Bullish_Engulfing'] == 1]
bearish_patterns = btc_data[btc_data['Bearish_Engulfing'] == 1]

In [ ]:
breakout_count = btc_data['Bullish_Engulfing'].sum()
print(f"Number of breakout signals: {breakout_count}")

In [ ]:
breakout_count = btc_data['Bearish_Engulfing'].sum()
print(f"Number of breakout signals: {breakout_count}")

In [ ]:
def detect_hammer(data):
    """
    Detects hammer candlestick patterns.
    A hammer has:
    1. A small body at the top
    2. A long lower shadow (at least 2x the body)
    3. Little or no upper shadow
    """

    # Calculate body and shadows
    data['Body'] = data['Price'] - data['Open']
    data['Upper_Shadow'] = data['High'] - data[['Open', 'Price']].max(axis=1)
    data['Lower_Shadow'] = data[['Open', 'Price']].min(axis=1) - data['Low']

    # Define hammer criteria
    data['Hammer'] = (
        # Body is small (using 0.3 times high-low range as threshold)
        (abs(data['Body']) <= 0.3 * (data['High'] - data['Low'])) &
        # Lower shadow is at least 2x the body length
        (data['Lower_Shadow'] >= 2 * abs(data['Body'])) &
        # Upper shadow is relatively small
        (data['Upper_Shadow'] <= 0.1 * (data['High'] - data['Low']))
    ).astype(int)

    # Clean up temporary columns
    data.drop(['Body', 'Upper_Shadow', 'Lower_Shadow'], axis=1, inplace=True)

    # Count consecutive hammers
    data['Consecutive_Hammer'] = (
        data['Hammer']
        .groupby((data['Hammer'] == 0).cumsum())
        .cumcount()
    )
    data.loc[data['Hammer'] == 0, 'Consecutive_Hammer'] = 0

    return data

# Apply hammer detection to the data
btc_data = detect_hammer(btc_data)

In [ ]:
hammer_count = btc_data['Hammer'].sum()
print(f"Total number of hammer patterns detected: {hammer_count}")

# Also get the number of cases where we had consecutive hammers
consecutive_hammers = btc_data[btc_data['Consecutive_Hammer'] > 0]
print(f"Number of cases with consecutive hammers: {len(consecutive_hammers)}")
print(f"Maximum number of consecutive hammers: {btc_data['Consecutive_Hammer'].max()}")

In [ ]:
def detect_three_white_soldiers(data, min_body_ratio=0.5):
    # Create candle body and shadow lengths
    data['Body'] = data['Price'] - data['Open']
    data['Upper_Shadow'] = data['High'] - data[['Price', 'Open']].max(axis=1)
    data['Lower_Shadow'] = data[['Price', 'Open']].min(axis=1) - data['Low']

    # Initialize Three White Soldiers pattern array
    three_white_soldiers = np.zeros(len(data))

    for i in range(2, len(data)):
        # Check if we have 3 consecutive bullish candles
        bullish_candles = (data['Body'].iloc[i] > 0 and
                          data['Body'].iloc[i-1] > 0 and
                          data['Body'].iloc[i-2] > 0)

        if bullish_candles:
            # Calculate total candle lengths
            candle_lengths = [
                data['High'].iloc[i] - data['Low'].iloc[i],
                data['High'].iloc[i-1] - data['Low'].iloc[i-1],
                data['High'].iloc[i-2] - data['Low'].iloc[i-2]
            ]

            # Calculate body ratios
            body_ratios = [
                data['Body'].iloc[i] / candle_lengths[0],
                data['Body'].iloc[i-1] / candle_lengths[1],
                data['Body'].iloc[i-2] / candle_lengths[2]
            ]

            # Check if each candle has a sufficient body ratio
            strong_bodies = all(ratio >= min_body_ratio for ratio in body_ratios)

            # Check if each candle opens within previous candle's body
            progressive_opens = (
                data['Open'].iloc[i] > data['Open'].iloc[i-1] and
                data['Open'].iloc[i-1] > data['Open'].iloc[i-2]
            )

            # Check if each candle closes higher than previous candle
            progressive_closes = (
                data['Price'].iloc[i] > data['Price'].iloc[i-1] and
                data['Price'].iloc[i-1] > data['Price'].iloc[i-2]
            )

            # If all conditions are met, mark as Three White Soldiers
            if strong_bodies and progressive_opens and progressive_closes:
                three_white_soldiers[i] = 1

    # Add the pattern to the dataframe
    data['Three_White_Soldiers'] = three_white_soldiers

    # Count the occurrences
    pattern_count = int(three_white_soldiers.sum())

    return pattern_count, data['Three_White_Soldiers']

# Apply the detection
pattern_count, three_white_soldiers = detect_three_white_soldiers(btc_data)
print(f"Number of Three White Soldiers patterns detected: {pattern_count}")

In [ ]:
def detect_three_black_crows(data, min_body_ratio=0.5):
    # Create candle body and shadow lengths
    data['Body'] = data['Price'] - data['Open']
    data['Upper_Shadow'] = data['High'] - data[['Price', 'Open']].max(axis=1)
    data['Lower_Shadow'] = data[['Price', 'Open']].min(axis=1) - data['Low']

    # Initialize Three Black Crows pattern array
    three_black_crows = np.zeros(len(data))

    for i in range(2, len(data)):
        # Check if we have 3 consecutive bearish candles
        bearish_candles = (data['Body'].iloc[i] < 0 and
                          data['Body'].iloc[i-1] < 0 and
                          data['Body'].iloc[i-2] < 0)

        if bearish_candles:
            # Calculate total candle lengths
            candle_lengths = [
                data['High'].iloc[i] - data['Low'].iloc[i],
                data['High'].iloc[i-1] - data['Low'].iloc[i-1],
                data['High'].iloc[i-2] - data['Low'].iloc[i-2]
            ]

            # Calculate body ratios
            body_ratios = [
                abs(data['Body'].iloc[i]) / candle_lengths[0],
                abs(data['Body'].iloc[i-1]) / candle_lengths[1],
                abs(data['Body'].iloc[i-2]) / candle_lengths[2]
            ]

            # Check if each candle has a sufficient body ratio
            strong_bodies = all(ratio >= min_body_ratio for ratio in body_ratios)

            # Check if each candle opens within previous candle's body
            progressive_opens = (
                data['Open'].iloc[i] < data['Open'].iloc[i-1] and
                data['Open'].iloc[i-1] < data['Open'].iloc[i-2]
            )

            # Check if each candle closes lower than previous candle
            progressive_closes = (
                data['Price'].iloc[i] < data['Price'].iloc[i-1] and
                data['Price'].iloc[i-1] < data['Price'].iloc[i-2]
            )

            # If all conditions are met, mark as Three Black Crows
            if strong_bodies and progressive_opens and progressive_closes:
                three_black_crows[i] = 1

    # Add the pattern to the dataframe
    data['Three_Black_Crows'] = three_black_crows

    # Count the occurrences
    pattern_count = int(three_black_crows.sum())

    return pattern_count, data['Three_Black_Crows']

# Apply the detection
pattern_count, three_black_crows = detect_three_black_crows(btc_data)
print(f"Number of Three Black Crows patterns detected: {pattern_count}")

##Join External Data Source

In [ ]:
#Left join gold_price_data, risk_free_data, spy500 to btc_data
btc_data = pd.merge(btc_data, gold_price_data, on='Date', how='left') #till 2025-02-03
btc_data = pd.merge(btc_data, risk_free_data, on='Date', how='left')
btc_data = pd.merge(btc_data, spy_data, on='Date', how='left') #till 2025-04-11
btc_data = pd.merge(btc_data, eth_data, on='Date', how='left') #from 2015-08-07
#As other external data have market closeure, all na filled by previous day close data

In [ ]:
btc_data.info()

In [ ]:
# Plot Bitcoin Volume over date
plt.figure(figsize=(12, 6))
plt.plot(btc_data['Date'], btc_data['Volume'])
plt.xlabel('Date')
plt.ylabel('Bitcoin Volume')
plt.ylim(bottom=-2)  # Set the lower limit to 0
plt.title('Bitcoin Volume Over Time')
plt.xticks(rotation=45)
plt.show()

In [ ]:
btc_data['Volume'].describe()

Since we can observe the spikes happened from 2018 to mid-2022, we try to fit the model avoiding the spikes here.

In [ ]:
# Find the min and max of date with volume greater than 1
btc_data[btc_data['Volume'] > 1]['Date'].min(), btc_data[btc_data['Volume'] > 1]['Date'].max()
# Print the min and max date with volume greater than 1
print("Min date with volume > 1:", btc_data[btc_data['Volume'] > 1]['Date'].min())
print("Max date with volume > 1:", btc_data[btc_data['Volume'] > 1]['Date'].max())


In [ ]:
# Drop the rows with date before 2022-05-09
btc_data = btc_data[btc_data['Date'] >= '2022-05-09']

In [ ]:
# Plot Bitcoin Volume over date
plt.figure(figsize=(12, 6))
plt.plot(btc_data['Date'], btc_data['Volume'])
plt.xlabel('Date')
plt.ylabel('Bitcoin Volume')
plt.ylim(bottom=-2)  # Set the lower limit to 0
plt.title('Bitcoin Volume Over Time')
plt.xticks(rotation=45)
plt.show()

## **Re-scale and Transformation**

**Yeo-Johnson Transformation** for Price, Change, Pct_Change

In [ ]:
# For Price, Change, 7D_Pct_Change, 2D_Pct_Change (use yeo-johnson)
from sklearn.preprocessing import PowerTransformer
import pandas as pd

# Initialize PowerTransformer for each column
power_transformer_price = PowerTransformer(method='yeo-johnson', standardize=False)  # Disable standardization
power_transformer_change = PowerTransformer(method='yeo-johnson', standardize=False)
power_transformer_7d_pct_change = PowerTransformer(method='yeo-johnson', standardize=False)
power_transformer_2d_pct_change = PowerTransformer(method='yeo-johnson', standardize=False)

# Apply the Yeo-Johnson transformation to each column
btc_data['YeoJohnson_Price'] = power_transformer_price.fit_transform(btc_data[['Price']])
btc_data['YeoJohnson_Change'] = power_transformer_change.fit_transform(btc_data[['Change']])
btc_data['YeoJohnson_7D_Pct_Change'] = power_transformer_7d_pct_change.fit_transform(btc_data[['Pct_Change_7D']])
btc_data['YeoJohnson_2D_Pct_Change'] = power_transformer_2d_pct_change.fit_transform(btc_data[['Pct_Change_2D']])

# Store lambda values for each column
lambdas = {
    'Price_lambda': power_transformer_price.lambdas_[0],
    'Change_lambda': power_transformer_change.lambdas_[0],
    'Pct_Change_lambda_7D': power_transformer_7d_pct_change.lambdas_[0],
    'Pct_Change_lambda_2D': power_transformer_2d_pct_change.lambdas_[0]
}

# Calculate mean and standard deviation for each transformed column
transformed_means = {
    'Price_mean': btc_data['YeoJohnson_Price'].mean(),
    'Change_mean': btc_data['YeoJohnson_Change'].mean(),
    'Pct_Change_mean_7D': btc_data['YeoJohnson_7D_Pct_Change'].mean(),
    'Pct_Change_mean_2D': btc_data['YeoJohnson_2D_Pct_Change'].mean()
}

transformed_stds = {
    'Price_std': btc_data['YeoJohnson_Price'].std(),
    'Change_std': btc_data['YeoJohnson_Change'].std(),
    'Pct_Change_std_7D': btc_data['YeoJohnson_7D_Pct_Change'].std(),
    'Pct_Change_std_2D': btc_data['YeoJohnson_2D_Pct_Change'].std()
}

# Print the collected lambdas, means, and standard deviations
print(f"Lambdas: {lambdas}")
print(f"Means: {transformed_means}")
print(f"Standard Deviations: {transformed_stds}")

### Step 2: Back-Transformation ###
# Recreate PowerTransformers with stored lambdas for back-transformation
power_transformer_price.lambdas_ = [lambdas['Price_lambda']]
power_transformer_change.lambdas_ = [lambdas['Change_lambda']]
power_transformer_7d_pct_change.lambdas_ = [lambdas['Pct_Change_lambda_7D']]
power_transformer_2d_pct_change.lambdas_ = [lambdas['Pct_Change_lambda_2D']]

In [ ]:
binary_variables = [
    "MACD_buy",  # 1 if MACD crosses above MACD_Signal, otherwise 0
    "MACD_sell",  # 1 if MACD crosses below MACD_Signal, otherwise 0
    "MACD_above_signal",  # 1 if MACD > MACD_Signal, otherwise 0
    "MACD_below_signal",  # 1 if MACD <= MACD_Signal, otherwise 0
    "BB_Buy",  # 1 if Price crosses below the Lower Bollinger Band, otherwise 0
    "BB_Sell",  # 1 if Price crosses above the Upper Bollinger Band, otherwise 0
    "Price_below_BB_lower",  # 1 if Price < BB_lower, otherwise 0
    "Price_above_BB_upper",  # 1 if Price > BB_upper, otherwise 0
    "VWAP_Buy",  # 1 if Price crosses above VWAP, otherwise 0
    "VWAP_Sell",  # 1 if Price crosses below VWAP, otherwise 0
    "RSI_12_Oversold",  # 1 if RSI_12 < 30, otherwise 0
    "RSI_12_Overbought",  # 1 if RSI_12 > 70, otherwise 0
    "RSI_Divergence",  # 1 if RSI_6 > RSI_12, otherwise 0
    "Positive_7D",  # 1 if Pct_Change_7D > 0, otherwise 0
    "Positive_2D",  # 1 if Pct_Change_2D > 0, otherwise 0
    "Higher_Low",  # 1 if current low > previous low, otherwise 0
    "Breakout",  # 1 if Price > Resistance_Level, otherwise 0
    "Ascending_Triangle_Breakout",  # Same as Breakout
    "Breakout_With_Volume",  # 1 if breakout with volume confirmation, otherwise 0
    "Ascending_Triangle_Breakout_With_Volume",  # Same as Breakout_With_Volume
    "ADX_Buy_Signal",  # 1 if ADX > 25 and Positive_DI > Negative_DI, otherwise 0
    "ADX_Sell_Signal",  # 1 if ADX > 25 and Negative_DI > Positive_DI, otherwise 0
    "Stochastic_Buy_Signal",  # 1 if Stochastic %K < 20 and %K > %D, otherwise 0
    "Stochastic_Sell_Signal",  # 1 if Stochastic %K > 80 and %K < %D, otherwise 0
    "Bullish_Engulfing",  # 1 if Bullish Engulfing pattern is detected, otherwise 0
    "Bearish_Engulfing",  # 1 if Bearish Engulfing pattern is detected, otherwise 0
    "Hammer",  # 1 if Hammer candlestick pattern is detected, otherwise 0
    "Three_White_Soldiers",  # 1 if Three White Soldiers pattern is detected, otherwise 0
    "Three_Black_Crows", # 1 if Three Black Crows pattern is detected, otherwise 0
    'Extreme_Fear', 'Extreme_Greed'
]

In [ ]:
continuous_variables = [
    'Price', 'Open', 'High', 'Low', 'Change',
    "RSI_6",  # Relative Strength Index with a 6-period length
    "RSI_12",  # Relative Strength Index with a 12-period length
    "EMA_14",  # Exponential Moving Average with a 14-period length
    "SMA_14",  # Simple Moving Average with a 14-period length
    "OBV",  # On-Balance Volume
    "MACD",  # Moving Average Convergence Divergence
    "MACD_Signal",  # Signal line of MACD
    "MACD_cum_buy",  # Cumulative count of consecutive days where MACD_above_signal is 1
    "MACD_cum_sell",  # Cumulative count of consecutive days where MACD_below_signal is 1
    "BB_upper",  # Upper Bollinger Band
    "BB_lower",  # Lower Bollinger Band
    "BB_cum_Buy",  # Cumulative count of consecutive days where Price_below_BB_lower is 1
    "BB_cum_Sell",  # Cumulative count of consecutive days where Price_above_BB_upper is 1
    "ATR",  # Average True Range
    "ATR_Ratio",  # ATR as a percentage of Price
    "Stop_Loss_Long",  # Stop-loss level for long trades
    "Stop_Loss_Short",  # Stop-loss level for short trades
    "VWAP",  # Volume Weighted Average Price
    "Support",  # Lowest low in the past 20 days
    "Resistance",  # Highest high in the past 20 days
    "Consecutive_RSI_12_Overbought",  # Cumulative count of consecutive days where RSI_12_Overbought is 1
    "Consecutive_RSI_12_Oversold",  # Cumulative count of consecutive days where RSI_12_Oversold is 1
    "Pct_Change_7D",  # Percentage price change over the next 7 days
    "Pct_Change_2D",  # Percentage price change over the next 2 days
    "MA50",  # 50-day moving average
    "MA200",  # 200-day moving average
    "Price_vs_MA50",  # Price relative to 50-day MA (%)
    "Price_vs_MA200",  # Price relative to 200-day MA (%)
    "MA_Distance_Ratio",  # Ratio of MA50 to MA200 distance
    "Avg_Volume_20",  # 20-day average volume
    "Volume_Ratio",  # Current volume / 20-day avg volume
    "OBV_Slope",  # 5-day slope of OBV
    "Resistance_Level",  # Exponential moving average of high price
    "Avg_Volume",  # 7-day rolling average of volume
    "Triangle_Completion",  # Rolling mean of Higher_Low
    "Triangle_Height",  # Height of the triangle as % of price
    "Triangle_Duration",  # Sum of Higher_Low over 7 days
    "Breakout_Strength",  # Volume-adjusted breakout strength
    "ADX",  # Average Directional Index
    "Positive_DI",  # Positive Directional Index
    "Negative_DI",  # Negative Directional Index
    "Consecutive_ADX_Buy",  # Consecutive days of ADX_Buy_Signal
    "Consecutive_ADX_Sell",  # Consecutive days of ADX_Sell_Signal
    "ADX_Strength",  # How far ADX is above the threshold
    "ADX_Trend_Duration",  # Consecutive days where ADX > 25
    "ADX_Trend_Direction",  # Difference between Positive_DI and Negative_DI
    "Stochastic_perK",  # %K line of the Stochastic Oscillator
    "Stochastic_perD",  # %D line of the Stochastic Oscillator
    "Consecutive_Stochastic_Buy",  # Consecutive days of Stochastic_Buy_Signal
    "Consecutive_Stochastic_Sell",  # Consecutive days of Stochastic_Sell_Signal
    "Consecutive_Hammer",  # Consecutive hammer patterns
    "SPY_Price",  # Price of SPY (S&P 500 ETF)
    "SPY_Volume",  # Trading volume of SPY
    "ETH_Price",  # Price of Ethereum (ETH)
    "ETH_Volume",  # Trading volume of Ethereum (ETH)
    "Gold_Price",  # Price of gold
    "Gold_Volume",  # Trading volume of gold
    "US_10Y",  # Yield of the US 10-year Treasury bond
    "Yield_Spread",  # Spread between different bond yields
    "YeoJohnson_Price",  # Yeo-Johnson transformed Price
    "YeoJohnson_Change",  # Yeo-Johnson transformed Change
    "YeoJohnson_7D_Pct_Change",  # Yeo-Johnson transformed Pct_Change_7D
    "YeoJohnson_2D_Pct_Change", # Yeo-Johnson transformed Pct_Change_2D
    'Fear_and_Greed_Index', 'Consecutive_Extreme_Fear', 'Consecutive_Extreme_Greed'
]

In [ ]:
# Set binary variables to int64
btc_data[binary_variables] = btc_data[binary_variables].astype("int64")

# Set continuous variables to float64
btc_data[continuous_variables] = btc_data[continuous_variables].astype("float64")

# Verify the data types
for column in btc_data.columns:
    print(f"{column}: {btc_data[column].dtype}")

In [ ]:
numerical_variable = btc_data.select_dtypes(include=['number'])
btc_data.columns

**Standardize all numeric variable**

store to scaled_btc_data

In [ ]:
# Define count columns
count_columns = [ 'MACD_Signal', 'MACD_buy', 'MACD_sell', 'MACD_above_signal', 'MACD_cum_buy', 'MACD_cum_sell', 'MACD_below_signal', 'BB_Buy', 'BB_Sell', 'Price_below_BB_lower', 'Price_above_BB_upper',
                 'BB_cum_Buy', 'BB_cum_Sell', 'VWAP_Buy', 'VWAP_Sell', 'Price_above_VWAP', 'Price_below_VWAP', 'VWAP_cum_Buy', 'VWAP_cum_Sell', 'RSI12_Oversold' ,'Consecutive_RSI_12_Oversold',
                 'RSI_12_Overbought', 'RSI_Divergence', 'Consecutive_RSI_12_Overbought', 'Ascending_Triangle_Breakout', 'Ascending_Triangle_Breakout_With_Volume', 'ADX_Buy_Signal', 'ADX_Sell_Signal', 'Consecutive_ADX_Buy',
                 'Consecutive_ADX_Sell', 'Stochastic_Buy_Signal', 'Stochastic_Sell_Signal', 'Consecutive_Stochastic_Buy', 'Consecutive_Stochastic_Sell', 'Bullish_Engulfing', 'Bearish_Engulfing', 'Hammer', 'Consecutive_Hammer',
                 'Three_white_soldiers', 'Three_Black_Crows', 'Fear_and_Greed_Index', 'Extreme_Fear', 'Extreme_Greed', 'Consecutive_Extreme_Fear', 'Consecutive_Extreme_Greed'
                 ]

In [ ]:
#Standardize all numeric variable
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Initialize the MinMaxScaler and StandardScaler
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Apply log transformation to 'Volume' to reduce the range
btc_data['Volume'] = np.log1p(btc_data['Volume'])  # log1p handles log(0) safely

# Apply Scaler
btc_data['Volume'] = min_max_scaler.fit_transform(btc_data[['Volume']])
btc_data['Volume'] = standard_scaler.fit_transform(btc_data[['Volume']])

# List of features for scaling
numeric_features = continuous_variables

# Fit and transform only the numeric features
scaled_features = standard_scaler.fit_transform(btc_data[numeric_features])

# Create a new DataFrame for scaled data
scaled_btc_data = btc_data.copy()  # Copy original DataFrame

# Replace numeric columns in the new DataFrame with scaled values
scaled_btc_data[numeric_features] = scaled_features

scaled_btc_data.tail()

In [ ]:
#numeric_features = [
  #'Price', 'Open', 'High', 'Low', 'Change', 'RSI_6', 'RSI_12', 'EMA_14',
  #'SMA_14', 'OBV', 'MACD', 'MACD_Signal','MACD_above_signal', 'MACD_below_signal','MACD_cum_buy', 'MACD_cum_sell',
  #'BB_lower', 'BB_upper','Price_below_BB_lower', 'Price_above_BB_upper', 'BB_cum_Buy', 'BB_cum_Sell',
  #'ATR', 'Stop_Loss_Long', 'Stop_Loss_Short', 'VWAP','Support', 'Resistance', 'YeoJohnson_Price',
  #'YeoJohnson_Change','YeoJohnson_7D_Pct_Change', 'YeoJohnson_2D_Pct_Change', 'Gold_Price', 'Gold_Volume',
  #'US_10Y', 'Yield_Spread','Triangle_Completion', 'Triangle_Height', 'Breakout_Strength',
  #'ADX_Strength', 'ATR_Ratio', 'Price_vs_MA50', 'Price_vs_MA200', 'MA_Distance_Ratio', 'OBV_Slope',
  #'Stochastic_perK', 'Stochastic_perD', 'SPY_Price', 'SPY_Volume', 'ETH_Price', 'ETH_Volume',
  #'Fear_and_Greed_Index', 'Extreme_Fear', 'Extreme_Greed', 'Consecutive_Extreme_Fear', 'Consecutive_Extreme_Greed']

In [ ]:
scaled_btc_data.describe().T

# **Please use scaled_btc_data for analysis**

In [ ]:
scaled_btc_data.info()

**Drop NA before Analysis**

In [ ]:
# Drop rows with NaN values in scaled_btc_data
scaled_btc_data = scaled_btc_data.dropna()

# Optionally, reset the index after dropping rows
scaled_btc_data = scaled_btc_data.reset_index(drop=True)

In [ ]:
scaled_btc_data.info()

In [ ]:
#Check data are consecutive
# Ensure 'Date' is sorted
scaled_btc_data = scaled_btc_data.sort_values('Date')

# Calculate the difference between consecutive dates
scaled_btc_data['Date_Diff'] = scaled_btc_data['Date'].diff().dt.days

# Check if all differences are 1 day
if (scaled_btc_data['Date_Diff'].iloc[1:] == 1).all():
    print("All dates are consecutive.")
else:
    print("There are missing dates.")
    print(scaled_btc_data[scaled_btc_data['Date_Diff'] > 1])  # Display rows with missing dates

# Drop the 'Date_Diff' column after the check
scaled_btc_data.drop(columns=['Date_Diff'], inplace=True)

In [ ]:
# Check the start and end dates
start_date = scaled_btc_data['Date'].min()
end_date = scaled_btc_data['Date'].max()

print(f"Data starts from: {start_date}")
print(f"Data ends at: {end_date}")

#Exploratory Data Analysis

In [ ]:
#Print the name of the columns
print(scaled_btc_data.columns)

In [ ]:
# Moving date to index
scaled_btc_data.set_index('Date', inplace=True)

target = 'YeoJohnson_7D_Pct_Change'
# target_binary = 'Positive_2D'
target_binary = 'Positive_7D'

exclude_list = ['Price', 'Open', 'High', 'Low', 'Change', 'Day_of_Week', 'Week_of_Year', 'Month','Quarter','Year',
                'VWAP', 'VWAP_Buy', 'VWAP_Sell', 'Resistance_Level', 'Low_Shifted', 'Higher_Low',
                'Breakout', 'Ascending_Triangle_Breakout', 'Avg_Volume', 'Breakout_With_Volume', 'Avg_Volume_20','Pct_Change_7D', 'Pct_Change_2D','Positive_2D','YeoJohnson_Price','YeoJohnson_Change','YeoJohnson_2D_Pct_Change']
# Select binary columns
binary_variables = scaled_btc_data.drop(columns=exclude_list+[target,target_binary]).select_dtypes(include=['int64']).columns.tolist()

# Identify 2-dimensional columns
for column in binary_variables:
    if len(scaled_btc_data[column].shape) != 1:  # Check if the column is not 1D
        print(f"Column '{column}' is 2-dimensional with shape {scaled_btc_data[column].shape}")

# Filter out 2D columns and remove target from predictors
binary_variables = [col for col in binary_variables if len(scaled_btc_data[col].shape) == 1 and col != target]


# Define predictors, all columns except the 7D_Pct_Change or YeoJohnson_7D_Pct_Change or 2D_Pct_Change or YeoJohnson_2D_Pct_Change
predictors = scaled_btc_data.drop(columns=[target, target_binary] + exclude_list + binary_variables, axis=1).columns.tolist()

# Display the first few rows of the DataFrame
scaled_btc_data.head()

In [ ]:
# Check the predictors
predictors

##Univariate Analysis

In [ ]:
import math

cols = predictors[1:] + [target]
ncols = 5
nrows = math.ceil(len(cols) / ncols)

fig, axes = plt.subplots(nrows, ncols, figsize=(20, 4 * nrows))
axes = axes.flatten()

for ax, col in zip(axes, cols):
    scaled_btc_data[col].hist(
        bins=30,
        ax=ax,
        edgecolor='black'
    )
    ax.set_title(col)
    ax.set_xlabel('')
    ax.set_ylabel('')

# turn off any unused subplots
for ax in axes[len(cols):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.show()

##Bivariate Analysis

In [ ]:
# Select numeric columns
numeric_variables = scaled_btc_data.select_dtypes(include=['float64']).columns.tolist()

# Identify 2-dimensional columns
for column in numeric_variables:
    if len(scaled_btc_data[column].shape) != 1:  # Check if the column is not 1D
        print(f"Column '{column}' is 2-dimensional with shape {scaled_btc_data[column].shape}")

# Filter out 2D columns and remove target from predictors
numeric_variables = [col for col in numeric_variables if len(scaled_btc_data[col].shape) == 1 and col != target]

# Drop rows with missing values
data_numeric = scaled_btc_data[numeric_variables + [target]].dropna()

# Define number of rows and columns for the subplot grid
n_cols = 3
n_rows = (len(numeric_variables) + n_cols - 1) // n_cols  # Ceiling division

# Create figure with subplots
fig = plt.figure(figsize=(15, 5 * n_rows))

# Create scatter plots
for i, column in enumerate(numeric_variables, 1):
    ax = plt.subplot(n_rows, n_cols, i)
    sns.scatterplot(data=data_numeric, x=column, y=target, ax=ax)
    ax.set_title(f"{column} vs {target}")
    ax.set_xlabel(column)
    ax.set_ylabel(target)

    # Rotate x-axis labels if they're too long
    plt.xticks(rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
# Create correlation matrix
correlation_matrix = data_numeric.corr()

plt.figure(figsize=(20, 20))

# Create heatmap with annotations
sns.heatmap(correlation_matrix,
            annot=True,
            cmap='coolwarm',
            center=0,
            square=True,
            fmt='.2f',
            linewidths=0.5,
            cbar_kws={"shrink": .5})

plt.title('Correlation Heatmap: Numeric Variables vs Price Change', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()

plt.show()

In [ ]:
# Drop rows with missing values
data_binary = scaled_btc_data[binary_variables + [target]].dropna()

# Define number of rows and columns for the subplot grid
n_cols = 3
n_rows = (len(binary_variables) + n_cols - 1) // n_cols  # Ceiling division

# Create figure with subplots
fig = plt.figure(figsize=(15, 5 * n_rows))

# Create scatter plots
for i, column in enumerate(binary_variables, 1):
    ax = plt.subplot(n_rows, n_cols, i)
    sns.scatterplot(data=data_binary, x=column, y=target, ax=ax)
    ax.set_title(f"{column} vs {target}")
    ax.set_xlabel(column)
    ax.set_ylabel(target)

    # Rotate x-axis labels if they're too long
    plt.xticks(rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
# Create correlation matrix
correlation_matrix = data_binary.corr()

plt.figure(figsize=(12, 10))

# Create heatmap with annotations
sns.heatmap(correlation_matrix,
            annot=True,
            cmap='coolwarm',
            center=0,
            square=True,
            fmt='.3f',
            linewidths=0.5,
            cbar_kws={"shrink": .5})

plt.title('Correlation Heatmap: Binary Variables vs Price Change', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()

plt.show()

# Model Building

- Time-Based Splitting
- Selecting predictors
- Interaction terms

In [ ]:
from itertools import chain, combinations

def gen_combinations(input):
    return sum([list(map(list, combinations(input, i))) for i in range(1,len(input) + 1)], [])

def gen_interactions(X,de,pname):
    s = gen_combinations(range(len(pname)))
    VX = []
    VarX = []
    for i in range(len(s)):
        if len(s[i]) <= de:
            VX.append(np.prod(X[:,s[i]],axis=1))
            VarX.append(list([pname[x] for x in s[i]]))
    VarX2 = []
    for x in VarX:
        if len(x) > 1:
            VarX2.append('_'.join(x))
        else:
            VarX2.append(x[0])
    VX = pd.DataFrame(np.array(VX).T,columns=VarX2)
    return VX, VarX2



In [ ]:
import pandas as pd
import itertools

# Define feature groups
feature_groups = {
    'Group 1': [
        'Negative_DI', 'Positive_DI', 'ADX_Buy_Signal', 'ADX_Sell_Signal', 'ADX_Strength',
        'ADX_Trend_Direction', 'ADX_Trend_Duration', 'ADX'
    ],
    'Group 2': [
        'Ascending_Triangle_Breakout_With_Volume', 'Triangle_Completion', 'Triangle_Duration', 'Triangle_Height'
    ],
    'Group 3': [
        'ATR_Ratio', 'ATR'
    ],
    'Group 4': [
        'BB_Buy', 'BB_cum_Buy', 'BB_cum_Sell', 'BB_lower', 'BB_Sell', 'BB_upper',
        'Price_above_BB_upper', 'Price_below_BB_lower'
    ],
    'Group 7': [
        'Consecutive_RSI_12_Overbought', 'Consecutive_RSI_12_Oversold', 'RSI_12_Overbought',
        'RSI_12_Oversold', 'RSI_12', 'RSI_6', 'RSI_Divergence'
    ],
    'Group 8': [
        'Consecutive_Stochastic_Buy', 'Consecutive_Stochastic_Sell', 'Stochastic_perD',
        'Stochastic_perK', 'Stochastic_Buy_Signal', 'Stochastic_Sell_Signal'
    ],
    'Group 11': [
        'MA_Distance_Ratio', 'MA200', 'MA50', 'MACD_above_signal', 'MACD_below_signal',
        'MACD_buy', 'MACD_cum_buy', 'MACD_cum_sell', 'MACD_sell', 'MACD_Signal',
        'MACD', 'Price_vs_MA200', 'Price_vs_MA50'
    ],
    'Group 12': [
        'OBV_Slope', 'OBV', 'Resistance', 'Support', 'Volume_Ratio', 'Volume'
    ],
    'Group 13': [
        'SPY_Price', 'SPY_Volume'
    ],
    'Group 14': [
        'US_10Y', 'Yield_Spread','Fear_and_Greed_Index', 'Extreme_Fear', 'Extreme_Greed', 'Consecutive_Extreme_Fear', 'Consecutive_Extreme_Greed'
    ],
    'Group 15': [
        'Gold_Price', 'Gold_Volume'
    ],
    'Group 16': [
        'ETH_Price', 'ETH_Volume'
    ]
}

# Define interaction rules (between groups)
interaction_rules = [
    ('Group 1', 'Group 2'),  # ADX x Triangle Patterns
    ('Group 3', 'Group 4'),  # ATR x Bollinger Bands
    ('Group 7', 'Group 8'),  # RSI x Stochastic Oscillator
    ('Group 11', 'Group 12'),  # MACD x Volume and Support/Resistance
    ('Group 14', 'Group 13'),  # Macro Indicators x SPY
    ('Group 15', 'Group 16')  # Gold x ETH
]

# Initialize an empty DataFrame for interaction terms
interaction_terms = pd.DataFrame(index=scaled_btc_data.index)

# Generate between-group interaction terms
for group_a, group_b in interaction_rules:
    print(f"Generating interactions between {group_a} and {group_b}...")
    features_a = feature_groups[group_a]
    features_b = feature_groups[group_b]
    for feature_a, feature_b in itertools.product(features_a, features_b):  # All combinations between groups
        interaction_name = f"{feature_a}_x_{feature_b}"
        interaction_terms[interaction_name] = scaled_btc_data[feature_a] * scaled_btc_data[feature_b]


interaction_list = interaction_terms.columns.tolist()
# Add interaction terms to the original DataFrame
scaled_btc_data_with_interactions = scaled_btc_data.join(interaction_terms)

scaled_btc_data_with_interactions

In [ ]:
'''
# Generate within-group interaction terms
for group_name, features in feature_groups.items():
    print(f"Generating within-group interactions for {group_name}...")
    for feature_a, feature_b in itertools.combinations(features, 2):  # All unique pairs within the group
        interaction_name = f"{feature_a}_x_{feature_b}"
        interaction_terms[interaction_name] = scaled_btc_data[feature_a] * scaled_btc_data[feature_b]
'''

In [ ]:
"""
# 1) Select and clean your base predictors '''
relevant_predictors = [
    'VWAP_Buy', 'MACD_buy', 'BB_Buy',
    'RSI_12_Oversold', 'RSI_Divergence',
    'VWAP', 'MACD', 'RSI_6', 'RSI_12', 'ATR'
]
df_base = scaled_btc_data[relevant_predictors].dropna()

# 2) Generate all up-to-2-way interactions
X       = df_base.values
pnames  = df_base.columns.tolist()
inter_df, inter_names = gen_interactions(X, de=2, pname=pnames)

# 3) Build a set for quick membership tests
excluded = { tuple(sorted(pair)) for pair in [
    ('VWAP_Buy', 'VWAP_Sell'),
    ('MACD_buy','MACD_sell'),
    ('BB_Buy','BB_Sell'),
]}

# 4) Keep only the interactions we want
keep_mask = [
    not (len(name.split('_')) == 2
         and tuple(sorted(name.split('_'))) in excluded)
    for name in inter_names
]
valid_names = [name for name, keep in zip(inter_names, keep_mask) if keep]

# 5) Subset and align with the original index
inter_df = inter_df.loc[:, valid_names]
inter_df.index = df_base.index

# 6) Add a suffix to interaction columns to avoid overlap
inter_df = inter_df.add_suffix('_interaction')

# Join back onto your full DataFrame
scaled_btc_data_with_interactions = scaled_btc_data.join(inter_df)
scaled_btc_data_with_interactions
"""

## Splitting the data into training, testing and validation datasets

###5102

In [ ]:
final_df = scaled_btc_data_with_interactions.copy()
X = final_df[predictors + interaction_list + binary_variables]  # Define predictors
y = final_df[target]  # Define target variable


X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')

In [ ]:
# Splitting data (70% train, 15% validation, 15% test)
train_size = int(len(final_df) * 0.7)
val_size = int(len(final_df) * 0.15)

# Time-based splits
X_train = X.iloc[:train_size]
y_train = y.iloc[:train_size]

X_val = X.iloc[train_size : train_size + val_size]
y_val = y.iloc[train_size : train_size + val_size]

X_test = X.iloc[train_size + val_size :]
y_test = y.iloc[train_size + val_size :]

###5104

In [ ]:
# Create a new dataframe for binary classification
final_df_2 = scaled_btc_data_with_interactions.copy()

# Define binary target


print(final_df_2[target_binary].value_counts())

# Define predictors for classification

final_df_2 = scaled_btc_data_with_interactions.copy()
X_2 = final_df_2[predictors + binary_variables + interaction_list]  # Independent variables
y_2 = final_df_2[target_binary]  # Target variable

y_2 = pd.to_numeric(y_2, errors='coerce')


# Time-based splitting (70% train, 15% validation, 15% test)
train_size = int(len(final_df_2) * 0.7)
val_size = int(len(final_df_2) * 0.15)

# Create the splits for binary classification
X_train_2 = X_2.iloc[:train_size]
y_train_2 = y_2.iloc[:train_size]

X_val_2 = X_2.iloc[train_size:train_size + val_size]
y_val_2 = y_2.iloc[train_size:train_size + val_size]

X_test_2 = X_2.iloc[train_size + val_size:]
y_test_2 = y_2.iloc[train_size + val_size:]

print(f"Binary classification data shape: {X_2.shape}")
print(f"Training set: {X_train_2.shape[0]} samples")
print(f"Validation set: {X_val_2.shape[0]} samples")
print(f"Test set: {X_test_2.shape[0]} samples")

## Defining functions for model performance evaluation

###5102

In [ ]:
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# function to compute MAPE
def mape_score(targets, predictions):
    return np.mean(np.abs(targets - predictions) / targets) * 100


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
    mape = mape_score(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf

# RMSE
def rmse(predictions, targets):
    return np.sqrt(((targets - predictions) ** 2).mean())


# MAPE
def mape(predictions, targets):
    return np.mean(np.abs((targets - predictions)) / targets) * 100


# MAE
def mae(predictions, targets):
    return np.mean(np.abs((targets - predictions)))

# To see the feature importance of variables in the final model
def feature_importances(model, feature_names, n=10):
    if isinstance(model,LinearRegression):
        importances = model.coef_
    else:
        importances = model.feature_importances_
    zipped = sorted(zip(feature_names, importances), key=lambda x: -x[1])
    for i, f in enumerate(zipped[:n]):
        print("%d: Feature: %s, %.3f" % (i+1, f[0], f[1]))


# Model Performance on test and train data
def model_pref(olsmodel, x_train, x_test, y_train,y_test):

    # Insample Prediction
    y_pred_train = olsmodel.predict(x_train)
    y_observed_train = y_train

    # Prediction on test data
    y_pred_test = olsmodel.predict(x_test)
    y_observed_test = y_test

    print(
        pd.DataFrame(
            {
                "Data": ["Train", "Test"],
                "RMSE": [
                    rmse(y_pred_train, y_observed_train),
                    rmse(y_pred_test, y_observed_test),
                ],
                "MAE": [
                    mae(y_pred_train, y_observed_train),
                    mae(y_pred_test, y_observed_test),
                ],
                "MAPE": [
                    mape(y_pred_train, y_observed_train),
                    mape(y_pred_test, y_observed_test),
                ],
            }
        )
    )

###5014

In [ ]:
# Function to evaluate classification models
def model_performance_classification(model, X, y):
    """Evaluate classification model performance with comprehensive metrics"""
    from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                               f1_score, roc_curve, auc, confusion_matrix)

    # Generate predictions
    y_pred = model.predict(X)

    # Calculate confusion matrix values
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

    # Get probability predictions if available
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y == list(set(y))[1], y_prob)
        roc_auc = auc(fpr, tpr)
    else:
        roc_auc = None

    # Return all metrics
    return {
        'accuracy': accuracy_score(y, y_pred),
        'precision': precision_score(y, y_pred),
        'recall': recall_score(y, y_pred),
        'f1': f1_score(y, y_pred),
        'error_rate': 1 - accuracy_score(y, y_pred),
        'specificity': tn / (tn + fp),
        'roc_auc': roc_auc
    }

##Model 1: Linear Regression

In [ ]:

def gen_regstring(varlist,Y):
    st = Y+' ~ '
    if len(varlist) > 0:
        st = st+' + '.join(varlist)
    else:
        st = st + '1'
    return st

def gen_reg(varlist, depvar, data):
    lm = smf.ols(formula= gen_regstring(varlist,depvar), data = data).fit()
    return lm


In [ ]:
model = gen_reg(X_train.columns.tolist(),target,final_df)
print(model.summary())

In [ ]:
model_performance_regression(model, X_train, y_train)

In [ ]:
model_performance_regression(model, X_test, y_test)

### Checking Model Assumptions

Model assumption 1: Mean of residuals should be 0

In [ ]:
residuals = model.resid
mean_residual = residuals.mean()
print(f"Mean of residuals: {mean_residual:.6f}")

Model assumption 2: No Heteroscedasticity

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
residuals = model.resid
exog = model.model.exog  # predictors including intercept

# Breusch-Pagan test
bp_test = het_breuschpagan(residuals, exog)

# Results
names = ['Lagrange multiplier statistic', 'p-value',
         'f-value', 'f p-value']
print(dict(zip(names, bp_test)))

In [ ]:
plt.scatter(model.fittedvalues, model.resid)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()

Model assumption 3: Linearity

In [ ]:
y_pred = model.predict(sm.add_constant(X_train))

# Plot
plt.scatter(y_train, y_pred)
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], color='red', linestyle='--')  # 45-degree line
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted')
plt.show()

Model assumption 4: Normality of the residuals

In [ ]:
sm.qqplot(model.resid, line='45')
plt.title('Q-Q plot of residuals')
plt.show()

###Checking for Autocorrelation

In [ ]:
from statsmodels.stats.stattools import durbin_watson
dw = durbin_watson(model.resid)
print(f"Durbin-Watson statistic: {dw:.4f}")

###Checking for Multicollinearity

In [ ]:
vif_data = pd.DataFrame()
vif_data["Variable"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i)
                   for i in range(X_train.shape[1])]

# 3. Print
sorted_vif = vif_data.sort_values("VIF", ascending=False)
sorted_vif

Drop VIF>10

In [ ]:
# count the number of variables is infinite
# Drop rows where the "VIF" column is infinite

inf_feats = sorted_vif[sorted_vif["VIF"].apply(np.isinf)]["Variable"].tolist()
X_reduce_inf = X_train.drop(columns=inf_feats)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def drop_high_vif(X: pd.DataFrame,
                  thresh: float = 10.0,
                  bulk_thresh: float = 1e7):
    X_current = X.copy()
    while True:
        # compute VIF for each column
        vif = pd.Series(
            [variance_inflation_factor(X_current.values, i)
             for i in range(X_current.shape[1])],
            index=X_current.columns
        )

        # 1) bulk-drop any with VIF > bulk_thresh
        bulk_feats = vif[vif > bulk_thresh].index.tolist()
        if bulk_feats:
            for feat in bulk_feats:
                print(f"Dropping '{feat}' (VIF={vif[feat]:.0f} > {bulk_thresh:.0f})")
            X_current = X_current.drop(columns=bulk_feats)
            continue

        # 2) drop highest VIF > thresh one at a time
        max_vif = vif.max()
        if max_vif <= thresh:
            break

        drop_feat = vif.idxmax()
        print(f"Dropping '{drop_feat}' (VIF={max_vif:.2f})")
        X_current = X_current.drop(columns=[drop_feat])

    return X_current

In [ ]:
vifLess10_df = drop_high_vif(X_reduce_inf)
vifLess10_list= vifLess10_df.columns.tolist()

In [ ]:
model_reduce_vif = gen_reg(vifLess10_list,target,final_df)
print(model_reduce_vif.summary())

Back Elimination

In [ ]:
import statsmodels.formula.api as smf

def backward_elimination(
    data,
    response: str,
    predictors: list[str],
    sl: float = 0.05,
    verbose: bool = True
) -> list[str]:
    selected = predictors.copy()
    while True:
        # Build formula like "y ~ x1 + x2 + x3"
        formula = f"{response} ~ {' + '.join(selected)}"
        model   = smf.ols(formula, data=data).fit()
        # Drop the intercept from consideration
        pvals   = model.pvalues.drop('Intercept', errors='ignore')
        max_p   = pvals.max()

        if max_p > sl:
            worst = pvals.idxmax()
            selected.remove(worst)
            if verbose:
                print(f"Dropping '{worst}' (p = {max_p:.3f})")
        else:
            break

    return selected


In [ ]:
beSelected = backward_elimination(final_df, target, vifLess10_list, sl=0.05)

In [ ]:
model_reduce =gen_reg(beSelected,target,final_df)
print(model_reduce.summary())

In [ ]:
model_performance_regression(model_reduce,X_train[beSelected] , y_train)

In [ ]:
model_performance_regression(model_reduce, X_test[beSelected], y_test)

In [ ]:
#Model assumption 1: Mean of residuals should be 0
residuals = model_reduce.resid
mean_residual = residuals.mean()
print(f"Mean of residuals: {mean_residual:.6f}")

In [ ]:
#Model assumption 2: No Heteroscedasticity
residuals = model_reduce.resid
exog = model_reduce.model.exog  # predictors including intercept

# Breusch-Pagan test
bp_test = het_breuschpagan(residuals, exog)

# Results
names = ['Lagrange multiplier statistic', 'p-value',
         'f-value', 'f p-value']
print(dict(zip(names, bp_test)))

In [ ]:
plt.scatter(model_reduce.fittedvalues, model_reduce.resid)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()

In [ ]:
#Model assumption 3: Linearity
y_pred = model_reduce.predict(sm.add_constant(X_train[beSelected]))

# Plot
plt.scatter(y_train, y_pred)
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], color='red', linestyle='--')  # 45-degree line
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted')
plt.show()

In [ ]:
#Model assumption 4: Normality of the residuals
sm.qqplot(model_reduce.resid, line='45')
plt.title('Q-Q plot of residuals')
plt.show()

stepwise selection

In [ ]:
import statsmodels.formula.api as smf

def stepwise_selection(
    data,
    response: str,
    predictors: list[str],
    p_enter: float = 0.05,
    p_remove: float = 0.05,
    verbose: bool = True
) -> list[str]:
    selected = []
    remaining = predictors.copy()
    step = 0

    while True:
        step += 1
        changed = False

        # --- Forward step
        best_pval = None
        best_candidate = None
        for cand in remaining:
            formula = f"{response} ~ " + " + ".join(selected + [cand])
            model = smf.ols(formula, data=data).fit()
            pval = model.pvalues.get(cand, None)
            if pval is not None and (best_pval is None or pval < best_pval):
                best_pval, best_candidate = pval, cand

        if best_candidate and best_pval <= p_enter:
            selected.append(best_candidate)
            remaining.remove(best_candidate)
            changed = True
            if verbose:
                print(f"Step {step}: Add    '{best_candidate}' (p = {best_pval:.4f})")

        # --- Backward step
        if selected:
            formula = f"{response} ~ " + " + ".join(selected)
            model = smf.ols(formula, data=data).fit()
            # drop the intercept from consideration
            pvals = model.pvalues.drop("Intercept", errors="ignore")
            worst_var = pvals.idxmax()
            worst_p  = pvals.max()
            if worst_p > p_remove:
                selected.remove(worst_var)
                remaining.append(worst_var)
                changed = True
                if verbose:
                    print(f"Step {step}: Drop   '{worst_var}' (p = {worst_p:.4f})")

        if not changed:
            break

    return selected


In [ ]:
steplist = stepwise_selection(final_df, target, vifLess10_list)
steplist

In [ ]:
model_step = gen_reg(steplist,target,final_df)
print(model_step.summary())


PCA

In [ ]:
from scipy.stats import zscore

def stat_PCA(df, pname, n_components=None):
    # Subset and compute correlation matrix
    data = df[pname]
    corr = data.corr()

    # Eigen decomposition
    eigvals, eigvecs = np.linalg.eig(corr)
    # Sort by descending eigenvalue
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    # Determine number of components
    p = len(pname)
    if n_components is None or n_components > p:
        n_components = p

    # Select top n_components
    eigvecs = eigvecs[:, :n_components]

    # Compute PCA scores
    Z = data.apply(zscore)
    scores = Z.dot(eigvecs)
    scores.columns = [f'P{i+1}' for i in range(n_components)]

    # Create loadings DataFrame
    loadings = pd.DataFrame(eigvecs,
                            index=pname,
                            columns=[f'P{i+1}' for i in range(n_components)])

    return scores, loadings


In [ ]:
pcscore_train,loadings_train = stat_PCA(final_df,X_train.drop(columns=binary_variables+interaction_list).columns.tolist(), n_components=10)
pcscore_train

In [ ]:
eigvals = (loadings_train ** 2).sum(axis=0)

# 2) turn them into proportions of total variance:
#    (for a correlation‐matrix PCA, sum of eigenvalues = n_variables)
prop_explained = eigvals / eigvals.sum()

print(prop_explained)

In [ ]:
loadings_train['Prop_Var_Explained'] = (loadings_train ** 2).sum(axis=1)
loadings_train

In [ ]:
# Ensure y_train is aligned with pcscore_train
# Convert pcscore_train and target to float64 to avoid complex numbers
pclist = pcscore_train.columns.tolist()

pcscore_train = pcscore_train.astype('float64')

pcscore_train['target'] = final_df[target].astype('float64')

# Generate the regression model
model_pca = gen_reg(pclist, 'target', pcscore_train)
print(model_pca.summary())

Lasso/ Ridge?

##Model 2: ANN

5102

In [ ]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

def ann_lin(x_train, y_train, x_test, y_test,
            size, max_iter=100, trace=False, n_try=5):
    best_train_mse = np.inf
    best_test_mse  = np.inf
    best_train_model = None
    best_test_model  = None

    for i in range(n_try):
        # create a regressor with one hidden layer of 'size' units
        model = MLPRegressor(hidden_layer_sizes=(size,),
                             activation='relu',
                             solver='adam',
                             max_iter=max_iter,
                             random_state=i)

        model.fit(x_train, y_train)

        # compute train MSE
        y_train_pred = model.predict(x_train)
        train_mse = mean_squared_error(y_train, y_train_pred)

        # compute test MSE
        y_test_pred = model.predict(x_test)
        test_mse = mean_squared_error(y_test, y_test_pred)

        if trace:
            print(f"Run {i+1}/{n_try} → train MSE: {train_mse:.4f}, test MSE: {test_mse:.4f}")

        # update best-by-train
        if train_mse < best_train_mse:
            best_train_mse = train_mse
            best_train_model = model

        # update best-by-test
        if test_mse < best_test_mse:
            best_test_mse = test_mse
            best_test_model = model

    return {
        'best_train_mse': best_train_mse,
        'best_test_mse':  best_test_mse,
        'best_train_model': best_train_model,
        'best_test_model':  best_test_model
    }


In [ ]:
model_ann = ann_lin(X_train, y_train, X_test, y_test,
            3, max_iter=10000, trace=False, n_try=5)

In [ ]:
model_ann

In [ ]:
model_performance_regression(model_ann['best_train_model'], X_train, y_train)

In [ ]:
model_performance_regression(model_ann['best_test_model'], X_test, y_test)

5104 ann

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error

def ann_cla(x_train, y_train, x_test, y_test,
            size, max_iter=100, trace=False, n_try=5):
    best_train_mse = np.inf
    best_test_mse  = np.inf
    best_train_model = None
    best_test_model  = None

    for i in range(n_try):
        model = MLPClassifier(
            hidden_layer_sizes=(size,),
            activation='relu',
            solver='adam',
            max_iter=max_iter,
            random_state=i
        )
        model.fit(x_train, y_train)

        # predict class labels (0/1)
        y_train_pred = model.predict(x_train)
        y_test_pred  = model.predict(x_test)

        # MSE on 0/1 labels = error rate
        train_mse = mean_squared_error(y_train, y_train_pred)
        test_mse  = mean_squared_error(y_test,  y_test_pred)

        if trace:
            print(f"Run {i+1}/{n_try} → train MSE: {train_mse:.4f}, test MSE: {test_mse:.4f}")

        # pick best by train
        if train_mse < best_train_mse:
            best_train_mse   = train_mse
            best_train_model = model

        # pick best by test
        if test_mse < best_test_mse:
            best_test_mse    = test_mse
            best_test_model  = model

    return {
        'best_train_mse':   best_train_mse,
        'best_test_mse':    best_test_mse,
        'best_train_model': best_train_model,
        'best_test_model':  best_test_model
    }


In [ ]:
model_ann_cla = ann_cla(X_train_2, y_train_2, X_test_2, y_test_2,
                8, max_iter=100, trace=False, n_try=5)

In [ ]:
model_ann_cla


In [ ]:
ann_model = model_ann_cla['best_test_model']

##Model 3: Random Forest (5102)

In [ ]:
# Build random forest model
regressor = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_leaf=100,
    max_features='log2',
    min_samples_split=100,
    bootstrap=True,
    oob_score=True,
    random_state=42,
    n_jobs=-1
)

# Fit on training data
regressor.fit(X_train, y_train)

Validation performance:     RMSE    MAE  R-squared  Adj. R-squared      MAPE
0 1.1382 0.9288    -0.6153          1.8947 -159.9778

In [ ]:
# Evaluate on validation set
val_performance = model_performance_regression(regressor, X_val, y_val)
print("Validation performance:", val_performance)


Final test performance:     RMSE    MAE  R-squared  Adj. R-squared    MAPE
0 0.8903 0.6983    -0.0437          1.5842 81.3601

In [ ]:
# Final evaluation on test set
final_test_performance_regressor = model_performance_regression(regressor, X_test, y_test)
print("\nFinal test performance:", final_test_performance_regressor)

###Feature Selection

In [ ]:
from sklearn.inspection import permutation_importance
# Calculate permutation importance on validation set
perm_result = permutation_importance(
    regressor, X_val, y_val,
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

# Get importance scores and sort them
importances = perm_result.importances_mean
indices = np.argsort(importances)[::-1]  # Sort in descending order

In [ ]:
# Plot top 15 feature importances
n_top = min(15, len(indices))
plt.figure(figsize=(12, 6))
plt.title('Top 15 Feature Importance', fontsize=15)
plt.bar(range(n_top), importances[indices[:n_top]], color='darkgreen')
plt.xticks(range(n_top), [X_train.columns[i] for i in indices[:n_top]], rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Print top features
print("\nTop 15 features:")
for i in range(n_top):
    print(f"{X_train.columns[indices[i]]}: {importances[indices[i]]:.4f}")

In [ ]:
# Select features with positive importance
threshold = 0  # Only include features that improve predictions
selected_features = [X_train.columns[i] for i in indices if importances[i] > threshold]
print(f"Selected {len(selected_features)} out of {X_train.shape[1]} features")
print("Selected features:", selected_features)

In [ ]:
# Create datasets with only selected features
X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]
X_test_selected = X_test[selected_features]

###Model Validation and Hyperparameter Tuning

Process time around 80 mins

In [ ]:
from tqdm.auto import tqdm

# Hyperparameter tuning
rf_tuned = RandomForestRegressor(random_state=42)

# Grid of parameters to choose from

parameters = {
'n_estimators': [100, 300],
'max_depth': [None, 10, 20],
'max_features': ['sqrt', 'log2', 0.7],
'min_samples_split': [2, 5],
'min_samples_leaf': [1, 2, 4],
'bootstrap': [True],
'criterion': ['squared_error'],
'random_state': [42],
'n_jobs': [-1],
'max_samples': [0.8],
}

'''parameters = {
    'n_estimators': [100, 120],
    'max_depth': [None, 15],
    'max_features': ['sqrt', 'log2', 0.7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
} # process time around 16 mins'''

'''parameters = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2', 0.7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True]
}'''

'''parameters = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 5, 10, 15],
    "max_features": ["sqrt", "log2", 0.8, 1],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}'''

'''parameters = {
    "n_estimators": [110, 120],
    "max_depth": [5, 7],
    "max_features": [0.8, 1]
}'''

'''parameters = {
    'n_estimators': [100, 200],
    'max_depth': [None, 15],
    'max_features': ['sqrt', 0.7]
}'''

# Use time series cross-validation
tscv = TimeSeriesSplit(n_splits=3)

# Calculate total parameter combinations for progress tracking
total_combinations = (
    len(parameters['n_estimators']) *
    len(parameters['max_depth']) *
    len(parameters['max_features']) *
    len(parameters['min_samples_split']) *
    len(parameters['min_samples_leaf']) *
    len(parameters['bootstrap']) *
    len(parameters['criterion']) *
    len(parameters['random_state']) *
    len(parameters['n_jobs']) *
    len(parameters['max_samples'])
)

print(f"Running grid search with {total_combinations} parameter combinations and {tscv.n_splits} CV splits")
print(f"Total model fits to perform: {total_combinations * tscv.n_splits}")

# Custom verbose callback to show progress
class TqdmCallback:
    def __init__(self, total):
        self.pbar = tqdm(total=total, desc="GridSearchCV Progress")
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.count += 1
        self.pbar.update(1)

    def close(self):
        self.pbar.close()

progress_callback = TqdmCallback(total_combinations * tscv.n_splits)

# Run the grid search with custom verbose callback
with tqdm(total=1, desc="Overall Progress") as pbar:
    grid_obj = GridSearchCV(
        rf_tuned,
        parameters,
        scoring='r2',
        cv=tscv,
        n_jobs=-1,
        verbose=2
    )

    # Fit model with progress tracking
    grid_obj.fit(X_train_selected, y_train)
    pbar.update(1)

# Get the best model
rf_tuned_regressor = grid_obj.best_estimator_
print("Best parameters:", grid_obj.best_params_)
tuned_val_performance = model_performance_regression(rf_tuned_regressor, X_val_selected, y_val)
print("Tuned model validation performance:", tuned_val_performance)

In [ ]:
# Train final model on combined training and validation data
X_train_val = pd.concat([X_train_selected, X_val_selected])
y_train_val = pd.concat([y_train, y_val])

final_rfmodel = RandomForestRegressor(**grid_obj.best_params_)
final_rfmodel.fit(X_train_val, y_train_val)

Final test performance:     RMSE    MAE  R-squared  Adj. R-squared     MAPE
0 1.2189 0.9298    -0.9563          8.5243 244.8741

In [ ]:
# Final evaluation on test set only
final_test_performance = model_performance_regression(final_rfmodel, X_test_selected, y_test)
print("\nFinal test performance:", final_test_performance)

In [ ]:
# Plot actual vs predicted values on test set
y_pred = final_rfmodel.predict(X_test_selected)
plt.figure(figsize=(14, 6))
plt.title('Bitcoin Price: Actual vs Predicted (Selected Features)', fontsize=15)
plt.plot(y_test.index, y_test, label='Actual', color='blue')
plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Bitcoin Price % Change', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Function to generate predictions with confidence intervals using bootstrap
def predict_with_confidence(model, X_data, n_bootstrap=1000):

    predictions = []

    # Get all individual tree predictions
    for tree in tqdm(model.estimators_, desc="Building confidence intervals"):
        predictions.append(tree.predict(X_data))

    # Convert to numpy array
    predictions = np.array(predictions)

    # Calculate mean prediction (overall prediction)
    mean_prediction = np.mean(predictions, axis=0)

    # Calculate confidence intervals
    lower_bound = np.percentile(predictions, 2.5, axis=0)
    upper_bound = np.percentile(predictions, 97.5, axis=0)

    return mean_prediction, lower_bound, upper_bound

In [ ]:
# Prediction with the model
prediction = final_rfmodel.predict(X_test_selected)[0]  #the first prediction in test set

mean_preds, lower_bounds, upper_bounds = predict_with_confidence(final_rfmodel, X_test_selected)
margin = (upper_bounds[0] - lower_bounds[0]) / 2

print(f"\nPredicted Bitcoin % change: {mean_preds[0]:.2f}% ± {margin:.2f}% based on model prediction with 95% confidence")

## Model 4: Classification Tree & Random Forest (5104)


Result:

- 2D: Basic Decision Tree performance: {'Accuracy': 0.5134615384615384, 'Precision': 0.41935483870967744, 'Recall': 0.05241935483870968, 'F1 Score': 0.0931899641577061}
- 7D:

In [ ]:
# Function to evaluate classification performance
def model_performance_classification(model, X, y):
    """Evaluate classification model performance with comprehensive metrics"""
    from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                               f1_score, roc_curve, auc, confusion_matrix)

    # Generate predictions
    y_pred = model.predict(X)

    # Calculate confusion matrix values
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

    # Get probability predictions if available
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y == list(set(y))[1], y_prob)
        roc_auc = auc(fpr, tpr)
    else:
        roc_auc = None

    # Return all metrics
    return {
        'accuracy': accuracy_score(y, y_pred),
        'precision': precision_score(y, y_pred),
        'recall': recall_score(y, y_pred),
        'f1': f1_score(y, y_pred),
        'error_rate': 1 - accuracy_score(y, y_pred),
        'specificity': tn / (tn + fp),
        'roc_auc': roc_auc
    }

In [ ]:
# Build Classification tree
dt_model_2 = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=30,
    min_samples_leaf=15,
    criterion='gini',
    class_weight='balanced',
    splitter='best',
    max_features='sqrt',
    min_impurity_decrease=0.0001,
    random_state=42
)

dt_model_2.fit(X_train_2, y_train_2)
dt_model_2.fit(X_train_2, y_train_2)



In [ ]:
# Visualize decision tree
plt.figure(figsize=(25, 20))
plot_tree(dt_model_2, feature_names=X_train_2.columns, filled=True,
          rounded=True, fontsize=10, class_names=['No Increase', 'Increase'])
plt.title('Basic Decision Tree for Bitcoin Price Direction', fontsize=15)
plt.tight_layout()
plt.show()

# Evaluate on validation
dt_val_performance = model_performance_classification(dt_model_2, X_val_2, y_val_2)
print("Basic Decision Tree performance:", dt_val_performance)

Validation performance: {'Accuracy': 0.47307692307692306, 'Precision': 0.47186147186147187, 'Recall': 0.8790322580645161, 'F1 Score': 0.6140845070422535}

In [ ]:
# Build random forest classifier
classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=8,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features='sqrt',
    bootstrap=True,
    criterion='gini',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0001,
    class_weight='balanced',
    max_samples=0.8,
    random_state=42,
    n_jobs=-1
)


with tqdm(total=1, desc="Fitting classifier") as pbar:
    classifier.fit(X_train_2, y_train_2)
    pbar.update(1)

# Evaluate on validation set
val_performance = model_performance_classification(classifier, X_val_2, y_val_2)
print("Validation performance:", val_performance)

# Calculate permutation importance on validation set with progress bar
print("\nCalculating feature importance...")
with tqdm(total=1, desc="Permutation importance") as pbar:
    perm_result = permutation_importance(
        classifier, X_val_2, y_val_2,
        n_repeats=10,
        random_state=42,
        n_jobs=-1
    )
    pbar.update(1)

# Get importance scores and sort them
importances = perm_result.importances_mean
indices = np.argsort(importances)[::-1]  # Sort in descending order

# Plot only top 15 feature importances
plt.figure(figsize=(12, 8))
plt.title('Top 15 Feature Importance for Direction Prediction (Permutation Method)', fontsize=15)

# Select top 15 features
top_n = 15
top_indices = indices[:top_n]
top_importances = importances[top_indices]
feature_names = [X_train_2.columns[i] for i in top_indices]

# Create the bar plot for top features
plt.bar(range(len(top_indices)), top_importances, color='darkblue')
plt.xticks(range(len(top_indices)), feature_names, rotation=90)
plt.tight_layout()
plt.show()

# Select features with positive importance
threshold = 0  # Only include features that improve predictions
selected_features = [X_train_2.columns[i] for i in indices if importances[i] > threshold]
print(f"Selected {len(selected_features)} out of {X_train_2.shape[1]} features")
print("Selected features:", selected_features)

# Create datasets with only selected features
X_train_2_selected = X_train_2[selected_features]
X_val_2_selected = X_val_2[selected_features]
X_test_2_selected = X_test_2[selected_features]

# Hyperparameter tuning for classification
rf_tuned = RandomForestClassifier(random_state=42)

# Grid of parameters specifically for classification
parameters = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2', 0.7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample', None]  # Important for imbalanced classes
}

# Use time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Calculate total parameter combinations
total_combinations = (
    len(parameters['n_estimators']) *
    len(parameters['max_depth']) *
    len(parameters['max_features']) *
    len(parameters['min_samples_split']) *
    len(parameters['min_samples_leaf']) *
    len(parameters['class_weight'])
)
total_fits = total_combinations * tscv.n_splits

# Run the grid search with progress tracking
print(f"\nStarting grid search with {total_combinations} parameter combinations")
print(f"Total model fits to perform: {total_fits}\n")

# Create a wrapper to track progress
class ProgressGridSearchCV(GridSearchCV):
    def fit(self, X, y):
        with tqdm(total=1, desc="Grid search progress") as pbar:
            result = super().fit(X, y)
            pbar.update(1)
        return result

grid_obj = ProgressGridSearchCV(
    rf_tuned,
    parameters,
    scoring='f1',  # F1 score is good for imbalanced classes
    cv=tscv,
    n_jobs=-1,     # Use all CPU cores
    verbose=0      # Disable default verbose since we're using tqdm
)

print("Starting hyperparameter search...")
grid_obj.fit(X_train_2_selected, y_train_2)
print("Grid search completed!")

# Get the best model
rf_tuned_classifier = grid_obj.best_estimator_
print("Best parameters:", grid_obj.best_params_)
tuned_val_performance = model_performance_classification(rf_tuned_classifier, X_val_2_selected, y_val_2)
print("Tuned model validation performance:", tuned_val_performance)

# Show confusion matrix
y_val_pred = rf_tuned_classifier.predict(X_val_2_selected)
cm = confusion_matrix(y_val_2, y_val_pred)
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Validation Set')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['No Increase', 'Increase'], rotation=45)
plt.yticks(tick_marks, ['No Increase', 'Increase'])
plt.tight_layout()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# Add text annotations
thresh = cm.max() / 2
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
plt.show()

# Train final model on combined training and validation data
print("\nTraining final model on combined training and validation data...")
X_train_val_2 = pd.concat([X_train_2_selected, X_val_2_selected])
y_train_val_2 = pd.concat([y_train_2, y_val_2])

final_rfmodel_2 = RandomForestClassifier(**grid_obj.best_params_, random_state=1)
with tqdm(total=1, desc="Training final model") as pbar:
    final_rfmodel_2.fit(X_train_val_2, y_train_val_2)
    pbar.update(1)

# Final evaluation on test set only once
final_test_performance = model_performance_classification(final_rfmodel_2, X_test_2_selected, y_test_2)
print("\nFinal test performance:", final_test_performance)

# Plot prediction probabilities on test set
y_test_proba = final_rfmodel_2.predict_proba(X_test_2_selected)[:, 1]  # Probability of class 1 (increase)
plt.figure(figsize=(14, 6))
plt.title('Bitcoin Direction: Probability of Price Increase', fontsize=15)
plt.plot(y_test_2.index, y_test_proba, label='Probability of Increase', color='blue')
plt.axhline(y=0.5, color='red', linestyle='--', label='Decision Threshold')
# Add actual outcomes as points
for idx, actual in zip(y_test_2.index, y_test_2):
    marker = 'o' if actual == 1 else 'x'
    color = 'green' if actual == 1 else 'red'
    plt.scatter(idx, y_test_proba[y_test_2.index.get_loc(idx)], marker=marker, color=color, s=50)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Probability', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Function to predict with confidence using prediction probabilities
def predict_with_confidence_classification(model, X_data, n_bootstrap=100):
    """Generate predictions with confidence intervals for classification probabilities"""
    all_proba = []

    # Get all individual tree predictions (probabilities) with progress bar
    for tree in tqdm(model.estimators_, desc="Building confidence intervals"):
        # For each tree, get the probability of class 1 (price increase)
        tree_proba = tree.predict_proba(X_data)[:, 1]
        all_proba.append(tree_proba)

    # Convert to numpy array
    all_proba = np.array(all_proba)

    # Calculate mean probability (overall prediction)
    mean_proba = np.mean(all_proba, axis=0)

    # Calculate confidence intervals
    lower_bound = np.percentile(all_proba, 2.5, axis=0)
    upper_bound = np.percentile(all_proba, 97.5, axis=0)

    # Predict labels based on mean probability
    pred_labels = (mean_proba >= 0.5).astype(int)

    return mean_proba, lower_bound, upper_bound, pred_labels

# Generate predictions with confidence intervals (already has progress bar)
print("\nGenerating predictions with confidence intervals...")
mean_probs, lower_bounds, upper_bounds, pred_labels = predict_with_confidence_classification(
    final_rfmodel_2, X_test_2_selected
)

# Display prediction for the first test sample
first_idx = 0
direction = "UP" if pred_labels[first_idx] == 1 else "DOWN"
confidence = mean_probs[first_idx] if pred_labels[first_idx] == 1 else 1 - mean_probs[first_idx]
margin = (upper_bounds[first_idx] - lower_bounds[first_idx]) / 2

print(f"\nPredicted Bitcoin direction: {direction}")
print(f"Confidence: {confidence*100:.2f}% ± {margin*100:.2f}% with 95% confidence interval")
print(f"Probability range: [{lower_bounds[first_idx]*100:.2f}%, {upper_bounds[first_idx]*100:.2f}%]")

In [ ]:
# Feature importance for final model
final_importances = pd.DataFrame({
    'Feature': selected_features,
    'Importance': final_rfmodel_2.feature_importances_
})
final_importances = final_importances.sort_values('Importance', ascending=False)

# Get only top 15 features
top_15_features = final_importances.head(15)

plt.figure(figsize=(12, 8))
plt.title('Final Model: Top 15 Features for Direction Prediction', fontsize=15)
plt.barh(range(len(top_15_features)), top_15_features['Importance'], color='darkblue')
plt.yticks(range(len(top_15_features)), top_15_features['Feature'])
plt.xlabel('Importance')
plt.gca().invert_yaxis()  # To have the highest importance at the top
plt.tight_layout()
plt.show()

print("\nTop 15 most important features for predicting price direction:")
print(final_importances.head(15))

##Model 5: Logistic Regression (5104)

Process time around 80 mins

In [ ]:
# Stepwise regression function
def stepwise_logistic_regression(X, y, initial_features=None, p_value_threshold=0.05,
                               direction='forward', max_iterations=50, max_features=15):

    all_features = X.columns.tolist()

    # Start with empty list for forward selection
    current_features = [] if initial_features is None else [f for f in initial_features if f in all_features]

    # Add constant for statsmodels
    X_with_const = sm.add_constant(X)

    # Initialize variables
    best_features = current_features.copy()
    iterations_log = []
    iteration = 0
    improved = True
    best_aic = float('inf')
    best_model = None

    print(f"Starting with {len(current_features)} features")
    if current_features:
        print(f"Initial features: {current_features}")

    # Start stepwise selection
    while improved and iteration < max_iterations and len(current_features) < max_features:
        improved = False
        iteration += 1
        print(f"\nIteration {iteration}:")

        # Define candidate features for forward selection
        forward_candidates = [f for f in all_features if f not in current_features]

        # Try adding one feature (forward selection)
        best_forward_feature = None
        best_forward_aic = float('inf')

        if direction in ['forward', 'both']:
            for feature in forward_candidates:
                candidate_features = current_features + [feature]
                try:
                    # Prepare features including constant
                    X_subset = X_with_const[['const'] + candidate_features]

                    # Fit model and get AIC
                    try:
                        model = sm.Logit(y, X_subset).fit(disp=0, method='lbfgs', maxiter=200)
                    except:
                        model = sm.Logit(y, X_subset).fit(disp=0, method='bfgs', maxiter=200)

                    aic = model.aic

                    # Check if p-value for the newly added feature is significant
                    p_value = model.pvalues.get(feature, 1.0)

                    if p_value <= p_value_threshold and aic < best_forward_aic:
                        best_forward_aic = aic
                        best_forward_feature = feature
                        best_forward_model = model
                        print(f"  Testing adding {feature}: AIC={aic:.2f}, p-value={p_value:.4f}")
                except Exception as e:
                    print(f"  Error testing {feature}: {str(e)[:60]}...")
                    continue

        # Check if we can add the feature
        if best_forward_feature and (best_forward_aic < best_aic):
            # Add the best feature
            print(f"Adding feature: {best_forward_feature}, AIC: {best_forward_aic:.2f}")
            current_features.append(best_forward_feature)
            best_aic = best_forward_aic
            best_model = best_forward_model
            improved = True
            iterations_log.append({
                'iteration': iteration,
                'action': 'add',
                'feature': best_forward_feature,
                'aic': best_forward_aic,
                'features': current_features.copy()
            })

        # If direction includes backward, check for insignificant features to remove
        if direction in ['backward', 'both'] and len(current_features) > 1 and best_model is not None:
            # Get p-values for all features
            p_values = {col: best_model.pvalues.get(col, 1.0) for col in current_features}

            # Find the most insignificant feature (if any)
            insignificant_features = [f for f in current_features if p_values.get(f, 0) > p_value_threshold]

            if insignificant_features:
                # Sort by p-value (most insignificant first)
                insignificant_features.sort(key=lambda f: p_values.get(f, 0), reverse=True)

                # Remove the most insignificant feature and refit
                feature_to_remove = insignificant_features[0]
                candidate_features = [f for f in current_features if f != feature_to_remove]

                try:
                    # Prepare features including constant
                    X_subset = X_with_const[['const'] + candidate_features]

                    # Fit model and get AIC
                    model = sm.Logit(y, X_subset).fit(disp=0, method='lbfgs', maxiter=200)
                    aic = model.aic

                    print(f"Testing removal of {feature_to_remove}: AIC={aic:.2f}, p-value={p_values.get(feature_to_remove, 1):.4f}")

                    # If removing improves AIC, remove it
                    if aic < best_aic:
                        print(f"Removing feature: {feature_to_remove}, AIC: {aic:.2f}")
                        current_features = candidate_features
                        best_aic = aic
                        best_model = model
                        improved = True
                        iterations_log.append({
                            'iteration': iteration,
                            'action': 'remove',
                            'feature': feature_to_remove,
                            'aic': aic,
                            'features': current_features.copy()
                        })
                except Exception as e:
                    print(f"Error removing {feature_to_remove}: {str(e)[:60]}...")

        # Update best features if improved
        if improved:
            best_features = current_features.copy()

        print(f"Current features ({len(current_features)}): {current_features}")
        print(f"Current AIC: {best_aic:.2f}")

    # Final model with best features
    if best_features:
        try:
            X_final = X_with_const[['const'] + best_features]
            final_model = sm.Logit(y, X_final).fit(disp=0, method='lbfgs', maxiter=200)
            return best_features, final_model, iterations_log
        except Exception as e:
            print(f"Error fitting final model: {str(e)}")
            return best_features, best_model, iterations_log

    return [], None, iterations_log


In [ ]:
# Calculate comprehensive metrics
def cal_metrics(y_true, y_pred):
    """Calculate detailed classification metrics"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    false_negative_rate = fn / (tp + fn) if (tp + fn) > 0 else 0
    error_rate = (fp + fn) / (tp + tn + fp + fn)

    return {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'Specificity': specificity,
        'F1_Score': f1,
        'False_Negative_Rate': false_negative_rate,
        'Error_Rate': error_rate
    }

In [ ]:
import time

# Build stepwise logistic regression
start_time = time.time()

initial_features = []  # Start with empty set

# Run stepwise regression
final_features, final_model, iterations_log = stepwise_logistic_regression(
    X_train_2,
    y_train_2,
    initial_features=initial_features,
    p_value_threshold=0.05,
    direction='both',  # Use both forward and backward steps
    max_iterations=100,
    max_features=300
)

elapsed_time = time.time() - start_time
print(f"\nStepwise regression completed in {elapsed_time:.2f} seconds")
print(f"Selected {len(final_features)} features out of {len(X_train_2.columns)} candidates")
print("Selected features:", final_features)

# Visualize the feature selection process
iterations_df = pd.DataFrame(iterations_log)
if not iterations_df.empty:
    plt.figure(figsize=(12, 6))
    plt.plot(iterations_df['iteration'], iterations_df['aic'], 'o-', color='blue')
    plt.xlabel('Iteration')
    plt.ylabel('AIC Score')
    plt.title('AIC Score Improvement During Stepwise Selection')
    plt.grid(True, alpha=0.3)

    # Annotate add/remove actions
    for i, row in iterations_df.iterrows():
        action_color = 'green' if row['action'] == 'add' else 'red'
        plt.annotate(
            f"{row['action']}: {row['feature'][:15]}",
            (row['iteration'], row['aic']),
            textcoords="offset points",
            xytext=(0, 10 if i % 2 == 0 else -20),
            ha='center',
            color=action_color,
            fontsize=8
        )

    plt.tight_layout()
    plt.show()

# Feature importance from the final model
if final_model is not None:
    # Get model coefficients (excluding constant)
    coefficients = final_model.params.drop('const', errors='ignore')

    # Calculate odds ratios and confidence intervals
    conf_int = final_model.conf_int()
    odds_ratios = np.exp(coefficients)
    odds_ratios_ci = np.exp(conf_int.drop('const', errors='ignore'))

    # Create a DataFrame with the results
    feature_importance = pd.DataFrame({
        'Feature': coefficients.index,
        'Coefficient': coefficients.values,
        'Odds_Ratio': odds_ratios.values,
        'CI_Lower': odds_ratios_ci[0].values,
        'CI_Upper': odds_ratios_ci[1].values,
        'P_Value': final_model.pvalues.drop('const', errors='ignore').values
    })

    # Sort by absolute coefficient value
    feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance = feature_importance.sort_values('Abs_Coefficient', ascending=False).reset_index(drop=True)

    # Plot feature importance
    plt.figure(figsize=(12, max(8, len(feature_importance) * 0.3)))

    # Color based on positive/negative impact
    colors = ['green' if c > 0 else 'red' for c in feature_importance['Coefficient']]

    # Create the bar plot
    plt.barh(
        feature_importance['Feature'],
        feature_importance['Coefficient'],
        color=colors
    )

    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.title('Feature Importance for Bitcoin Price Direction')
    plt.xlabel('Coefficient (Log Odds)')
    plt.ylabel('Feature')
    plt.grid(True, axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Print detailed feature importance table
    print("\nFeature Importance:")
    pd.set_option('display.max_rows', None)
    print(feature_importance[['Feature', 'Coefficient', 'Odds_Ratio', 'P_Value', 'CI_Lower', 'CI_Upper']])

    # Print model summary
    print("\nFinal Model Summary:")
    print(final_model.summary())

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report

# Evaluate on validation set
print("\nEvaluating on validation set...")

# Add constant for predictions
X_val_selected = sm.add_constant(X_val_2[final_features])

# Get validation predictions
val_probs = final_model.predict(X_val_selected)
val_preds = (val_probs > 0.5).astype(int)


# Calculate detailed validation metrics
val_metrics = cal_metrics(y_val_2, val_preds)
val_auc = roc_auc_score(y_val_2, val_probs)

print("\nValidation Set Performance:")
print(f"Accuracy: {val_metrics['Accuracy']:.4f}")
print(f"AUC: {val_auc:.4f}")
print(f"Recall (Sensitivity): {val_metrics['Recall']:.4f}")
print(f"Specificity: {val_metrics['Specificity']:.4f}")
print(f"Precision: {val_metrics['Precision']:.4f}")
print(f"F1 Score: {val_metrics['F1_Score']:.4f}")
print(f"False Negative Rate: {val_metrics['False_Negative_Rate']:.4f}")
print(f"Error Rate: {val_metrics['Error_Rate']:.4f}")
print("\nClassification Report:")
print(classification_report(y_val_2, val_preds))

# Plot confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val_2, val_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix (Validation)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks([0.5, 1.5], ['Decrease/No Change', 'Increase'])
plt.yticks([0.5, 1.5], ['Decrease/No Change', 'Increase'])
plt.tight_layout()
plt.show()

# Find optimal threshold
print("\nFinding optimal threshold...")
# Calculate metrics at different thresholds
thresholds = np.arange(0.01, 1, 0.01)
accuracy_scores = []
f1_scores = []
precision_scores = []
recall_scores = []
specificity_scores = []

for threshold in thresholds:
    y_pred = (val_probs >= threshold).astype(int)
    metrics = cal_metrics(y_val_2, y_pred)
    accuracy_scores.append(metrics['Accuracy'])
    f1_scores.append(metrics['F1_Score'])
    precision_scores.append(metrics['Precision'])
    recall_scores.append(metrics['Recall'])
    specificity_scores.append(metrics['Specificity'])

# Plot metrics vs threshold
plt.figure(figsize=(12, 8))
plt.plot(thresholds, accuracy_scores, 'b-', label='Accuracy')
plt.plot(thresholds, f1_scores, 'g-', label='F1 Score')
plt.plot(thresholds, precision_scores, 'r-', label='Precision')
plt.plot(thresholds, recall_scores, 'y-', label='Recall (Sensitivity)')
plt.plot(thresholds, specificity_scores, 'm-', label='Specificity')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Metrics vs. Threshold')
plt.grid(True, alpha=0.3)
plt.legend()

# Find optimal thresholds for different metrics
opt_acc_threshold = thresholds[np.argmax(accuracy_scores)]
opt_f1_threshold = thresholds[np.argmax(f1_scores)]

# Find threshold where precision and recall are closest (balanced)
pr_diff = np.abs(np.array(precision_scores) - np.array(recall_scores))
opt_pr_threshold = thresholds[np.argmin(pr_diff)]

plt.axvline(x=0.5, color='grey', linestyle='--', label='Default (0.5)')
plt.axvline(x=opt_acc_threshold, color='blue', linestyle='--',
            label=f'Optimal Accuracy ({opt_acc_threshold:.2f})')
plt.axvline(x=opt_f1_threshold, color='green', linestyle='--',
            label=f'Optimal F1 ({opt_f1_threshold:.2f})')
plt.axvline(x=opt_pr_threshold, color='purple', linestyle='--',
            label=f'Balanced P-R ({opt_pr_threshold:.2f})')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Optimal threshold for accuracy: {opt_acc_threshold:.2f}")
print(f"Optimal threshold for F1 score: {opt_f1_threshold:.2f}")
print(f"Balanced precision-recall threshold: {opt_pr_threshold:.2f}")

# Choose the threshold to use (you might prefer F1 for imbalanced data)
chosen_threshold = opt_f1_threshold
print(f"Using threshold: {chosen_threshold:.2f}")

In [ ]:
# Retrain on combined train+validation data with selected features
print("\nRetraining on combined training and validation data...")
X_train_val = pd.concat([X_train_2[final_features], X_val_2[final_features]])
y_train_val = pd.concat([y_train_2, y_val_2])

# Add constant for statsmodels
X_train_val_sm = sm.add_constant(X_train_val)

# Fit final statsmodels model
try:
    final_combined_model = sm.Logit(y_train_val, X_train_val_sm).fit(
        disp=0,
        method='lbfgs',
        maxiter=200
    )
    print("\nFinal Combined Model Summary:")
    print(final_combined_model.summary())
except Exception as e:
    print(f"Error fitting combined model: {str(e)}")
    print("Using original model instead")
    final_combined_model = final_model

In [ ]:
# Add constant explicitly
X_test_selected = sm.add_constant(X_test_2[final_features].copy())

# Ensure test data columns exactly match model's expected columns
model_columns = final_combined_model.params.index
X_test_selected_lr = pd.DataFrame(index=X_test_2.index)
X_test_selected_lr['const'] = 1.0

# Add each feature in the exact order expected by the model
for col in model_columns:
    if col != 'const':
        if col in X_test_2.columns:
            X_test_selected_lr[col] = X_test_2[col]
        else:
            print(f"Missing column in test data: {col}")
            # Fill with zeros as a fallback
            X_test_selected_lr[col] = 0

# Verify the alignment
print(f"Test data shape: {X_test_selected_lr.shape}")
print(f"Model params shape: {final_combined_model.params.shape}")
print(f"Columns match: {list(X_test_selected_lr.columns) == list(model_columns)}")

In [ ]:
# Evaluate on test set
print("\nEvaluating on test set...")

# Get test predictions
test_probs = final_combined_model.predict(X_test_selected_lr)
test_preds = (test_probs >= chosen_threshold).astype(int)

# Calculate detailed test metrics
test_metrics = cal_metrics(y_test_2, test_preds)
test_auc = roc_auc_score(y_test_2, test_probs)

print("\nTest Set Performance:")
print(f"Accuracy: {test_metrics['Accuracy']:.4f}")
print(f"AUC: {test_auc:.4f}")
print(f"Recall (Sensitivity): {test_metrics['Recall']:.4f}")
print(f"Specificity: {test_metrics['Specificity']:.4f}")
print(f"Precision: {test_metrics['Precision']:.4f}")
print(f"F1 Score: {test_metrics['F1_Score']:.4f}")
print(f"False Negative Rate: {test_metrics['False_Negative_Rate']:.4f}")
print(f"Error Rate: {test_metrics['Error_Rate']:.4f}")
print("\nClassification Report:")
classification_report(y_test_2, test_preds)

# Plot confusion matrix for test set
plt.figure(figsize=(8, 6))
cm_test = confusion_matrix(y_test_2, test_preds)
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix (Test)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks([0.5, 1.5], ['Decrease/No Change', 'Increase'])
plt.yticks([0.5, 1.5], ['Decrease/No Change', 'Increase'])
plt.tight_layout()
plt.show()

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test_2, test_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Bitcoin Price Direction Prediction')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

===== FINAL MODEL SUMMARY =====
Selected 21 features: ['RSI_12_Overbought_x_RSI_12', 'ADX_Trend_Duration_x_Triangle_Completion', 'Yield_Spread', 'Yield_Spread_x_Fear_and_Greed_Index', 'MACD_sell_x_Volume', 'Consecutive_RSI_12_Overbought_x_RSI_Divergence', 'ADX_x_Triangle_Duration', 'US_10Y_x_Consecutive_Extreme_Greed', 'MACD_cum_sell_x_Volume_Ratio', 'MACD_cum_sell', 'MACD_cum_buy_x_MACD', 'ADX_Trend_Direction_x_ADX_Trend_Duration', 'ADX_Strength_x_ADX_Trend_Duration', 'OBV', 'MACD_cum_buy_x_Volume', 'ADX_Buy_Signal_x_ADX_Trend_Direction', 'Price_vs_MA200_x_OBV', 'MACD_x_Price_vs_MA50', 'Yield_Spread_x_Extreme_Fear', 'Triangle_Height', 'ADX_Strength_x_Ascending_Triangle_Breakout_With_Volume']

Validation Performance:
Accuracy: 0.5333
AUC: 0.5933
Precision: 0.4699
Recall (Sensitivity): 0.6000
Specificity: 0.4824
F1 Score: 0.5270
False Negative Rate: 0.4000
Error Rate: 0.4667

Test Performance:
Accuracy: 0.6424
AUC: 0.5000
Precision: 0.6424
Recall (Sensitivity): 1.0000
Specificity: 0.0000
F1 Score: 0.7823
False Negative Rate: 0.0000
Error Rate: 0.3576
Optimal threshold: 0.18
===============================

In [ ]:
# Print comprehensive final summary
print("\n===== FINAL MODEL SUMMARY =====")
print(f"Selected {len(final_features)} features: {final_features}")

print("\nValidation Performance:")
print(f"Accuracy: {val_metrics['Accuracy']:.4f}")
print(f"AUC: {val_auc:.4f}")
print(f"Precision: {val_metrics['Precision']:.4f}")
print(f"Recall (Sensitivity): {val_metrics['Recall']:.4f}")
print(f"Specificity: {val_metrics['Specificity']:.4f}")
print(f"F1 Score: {val_metrics['F1_Score']:.4f}")
print(f"False Negative Rate: {val_metrics['False_Negative_Rate']:.4f}")
print(f"Error Rate: {val_metrics['Error_Rate']:.4f}")

print("\nTest Performance:")
print(f"Accuracy: {test_metrics['Accuracy']:.4f}")
print(f"AUC: {test_auc:.4f}")
print(f"Precision: {test_metrics['Precision']:.4f}")
print(f"Recall (Sensitivity): {test_metrics['Recall']:.4f}")
print(f"Specificity: {test_metrics['Specificity']:.4f}")
print(f"F1 Score: {test_metrics['F1_Score']:.4f}")
print(f"False Negative Rate: {test_metrics['False_Negative_Rate']:.4f}")
print(f"Error Rate: {test_metrics['Error_Rate']:.4f}")

print(f"\nOptimal threshold: {chosen_threshold:.2f}")
print("===============================")

#Model Performance Comparison

##5102

In [ ]:
# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
    mape = mape_score(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf

In [ ]:
# Linear regression model - baseline
baseline_reg_perf_test = model_performance_regression(model, X_test, y_test)
baseline_reg_perf_test

In [ ]:
# Linear regression model - Reduced model 1 (calculated VIF)
model_reduce_vif_perf_test = model_performance_regression(model_reduce_vif, X_test[vifLess10_list], y_test)
model_reduce_vif_perf_test

In [ ]:
# Linear regression model - Reduced model 2 (backward elimination)
model_reduce_perf_test = model_performance_regression(model_reduce, X_test[beSelected], y_test)
model_reduce_perf_test

In [ ]:
# Linear regression model - Reduced model 3 (stepwise regression)
model_step_perf_test = model_performance_regression(model_step, X_test[steplist], y_test)
model_step_perf_test

In [ ]:
# Linear regression model - Reduced model 4 (PCA)
pca_perf_test = model_performance_regression(model_pca, X_test, y_test)
pca_perf_test

In [ ]:
# ANN
ann_perf_test = model_performance_regression(model_ann['best_test_model'], X_test_ann, y_test)
ann_perf_test

In [ ]:
# Random Forest
regressor_perf_test = model_performance_regression(regressor, X_test, y_test)
regressor_perf_test

In [ ]:
# Tuned Random Forest Regressor
rf_tuned_regressor_perf_test = model_performance_regression(rf_tuned_regressor, X_test_selected, y_test)
rf_tuned_regressor_perf_test

In [ ]:
models_test_comp_df = pd.concat(
    [
        baseline_reg_perf_test.T,
        model_reduce_vif_perf_test.T,
        model_reduce_perf_test.T,
        model_step_perf_test.T,
        pca_perf_test.T,
        ann_perf_test.T,
        regressor_perf_test.T,
        rf_tuned_regressor_perf_test.T
    ],
    axis=1,
)

models_test_comp_df.columns = [
    "Baseline Linear Regression with all predictors",
    "Reduced model 1 (calculated VIF)",
    "Reduced model 2 (backward elimination)",
    "Reduced model 3 (stepwise regression)",
    "Reduced model 4 (PCA)",
    "ANN",
    "Random Forest regressor",
    "Tuned Random Forest Regressor"]

print("Test performance comparison:")
models_test_comp_df

##5104

In [ ]:
# Function to evaluate model performance
def evaluate_all_models(models, X_test_dict, y_test, model_names):
    """
    Parameters:
    -----------
    models : list
        List of trained model objects
    X_test_dict : dict
        Dictionary mapping model names to their test features
    y_test : array-like
        Target values
    model_names : list
        List of model names corresponding to the models

    Returns:
    --------
    metrics_df : DataFrame
        DataFrame containing performance metrics for all models
    """
    # First ensure y_test is binary (0/1)
    print(f"Original y_test shape: {y_test.shape}, dtype: {y_test.dtype}")
    print(f"Unique values in y_test: {np.unique(y_test)}")

    # Convert y_test to binary if needed
    if not np.array_equal(y_test, y_test.astype(bool).astype(int)):
        print("Converting y_test to binary (0/1)")
        y_test_binary = (y_test > 0.5).astype(int)
    else:
        y_test_binary = y_test.copy()

    print(f"After conversion - unique values in y_test: {np.unique(y_test_binary)}")

    metrics = {
        'Accuracy': [],
        'Error_Rate': [],
        'Precision': [],
        'Recall': [],
        'F1_Score': [],
        'ROC_AUC': [],
        'Specificity': []
    }

    all_predictions = {}
    all_probabilities = {}

    successful_models = []

    for i, (model, name) in enumerate(zip(models, model_names)):
        print(f"\nProcessing model: {name}")

        # Get the appropriate X_test for this model
        if name in X_test_dict:
            X_test_model = X_test_dict[name]
        else:
            print(f"Warning: No specific X_test found for {name}. Using first available.")
            X_test_model = list(X_test_dict.values())[0]

        try:
            # Make predictions
            y_pred_raw = model.predict(X_test_model)

            # Convert predictions to binary if needed
            if not np.array_equal(y_pred_raw, y_pred_raw.astype(bool).astype(int)):
                print(f"Converting predictions for {name} to binary (0/1)")
                y_pred = (y_pred_raw > 0.5).astype(int)
            else:
                y_pred = y_pred_raw

            # Try to get probability scores
            try:
                y_prob = model.predict_proba(X_test_model)[:, 1]
            except:
                print(f"No predict_proba for {name}, using alternative approach")
                # If predict_proba is not available, use predictions as probabilities
                if np.max(y_pred_raw) > 1 or np.min(y_pred_raw) < 0:
                    # For regression outputs, use sigmoid transformation
                    y_prob = 1 / (1 + np.exp(-y_pred_raw))
                else:
                    y_prob = y_pred_raw

            # Calculate confusion matrix elements
            cm = confusion_matrix(y_test_binary, y_pred)
            tn, fp, fn, tp = cm.ravel()

            # Calculate metrics
            accuracy = accuracy_score(y_test_binary, y_pred)
            precision = precision_score(y_test_binary, y_pred)
            recall = recall_score(y_test_binary, y_pred)
            f1 = f1_score(y_test_binary, y_pred)

            # ROC curve and AUC
            fpr, tpr, _ = roc_curve(y_test_binary, y_prob)
            roc_auc = auc(fpr, tpr)

            # Specificity
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

            # Store results
            metrics['Accuracy'].append(accuracy)
            metrics['Error_Rate'].append(1 - accuracy)
            metrics['Precision'].append(precision)
            metrics['Recall'].append(recall)
            metrics['F1_Score'].append(f1)
            metrics['ROC_AUC'].append(roc_auc)
            metrics['Specificity'].append(specificity)

            all_predictions[name] = y_pred
            all_probabilities[name] = y_prob
            successful_models.append(name)

            print(f"Successfully evaluated {name}")

        except Exception as e:
            print(f"Error processing model {name}: {e}")
            # Skip this model
            continue

    # Check if we have evaluated any models
    if not successful_models:
        print("No models were successfully evaluated!")
        return None

    # Create DataFrame for metrics with only successful models
    metrics_df = pd.DataFrame({k: v for k, v in metrics.items()}, index=successful_models)

    # 1. Metrics Table
    plt.figure(figsize=(15, 4))
    sns.heatmap(metrics_df, annot=True, cmap='YlGnBu', fmt='.3f',
                center=0.5, vmin=0, vmax=1)
    plt.title('Model Performance Comparison')
    plt.tight_layout()
    plt.show()

    # 2. ROC Curves
    plt.figure(figsize=(10, 8))
    for name in successful_models:
        fpr, tpr, _ = roc_curve(y_test_binary, all_probabilities[name])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.3f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves Comparison')
    plt.legend(loc="lower right")
    plt.show()

    # 3. Confusion Matrices with percentages
    fig, axes = plt.subplots(1, len(successful_models), figsize=(5*len(successful_models), 4))
    if len(successful_models) == 1:
        axes = [axes]

    for ax, name in zip(axes, successful_models):
        cm = confusion_matrix(y_test_binary, all_predictions[name])
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        # Plot both counts and percentages
        sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap='Blues')
        sns.heatmap(cm_normalized, annot=True, fmt='.2%', ax=ax, cmap='Blues', alpha=0.3)
        ax.set_title(f'Confusion Matrix\n{name}\n(count and percentage)')
        ax.set_ylabel('True Label')
        ax.set_xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

    # 4. Print detailed reports
    for name in successful_models:
        print(f"\nDetailed Performance Report for {name}:")
        print("-" * 50)
        print("Classification Report:")
        print(classification_report(y_test_binary, all_predictions[name]))

        # Calculate additional metrics for detailed reporting
        tn, fp, fn, tp = confusion_matrix(y_test_binary, all_predictions[name]).ravel()
        print("\nAdditional Metrics:")
        print(f"Error Rate: {(1 - accuracy_score(y_test_binary, all_predictions[name])):.3f}")
        print(f"Specificity: {tn/(tn+fp) if (tn+fp) > 0 else 0:.3f}")

    return metrics_df

In [ ]:
# Create a dictionary mapping model names to their corresponding test data
X_test_dict = {
    'ANN': X_test_2,
    'Decision Tree': X_test_2,
    'Random Forest': X_test_2,
    'Tuned Random Forest': X_test_2_selected,
    'Logistic Regression': X_test_selected_lr
}

# Set up the models list with the extracted ANN model
models = [ann_model, dt_model_2, classifier, final_rfmodel_2, final_combined_model]
model_names = ['ANN', 'Decision Tree', 'Random Forest', 'Tuned Random Forest', 'Logistic Regression']

# Evaluate all models using the dictionary of test datasets
metrics_df = evaluate_all_models(models, X_test_dict, y_test_2, model_names)

#Final Model Evaluation and Backtesting

Final Model selected: pending

## Back Transformation After prediction (for interpretation)

In [ ]:
# Recreate PowerTransformers with stored lambdas for back-transformation
power_transformer_price.lambdas_ = [lambdas['Price_lambda']]
power_transformer_change.lambdas_ = [lambdas['Change_lambda']]
power_transformer_7d_pct_change.lambdas_ = [lambdas['Pct_Change_lambda_7D']]
power_transformer_2d_pct_change.lambdas_ = [lambdas['Pct_Change_lambda_2D']]

# Step 1: De-standardize the standardized values to get Yeo-Johnson transformed values
scaled_btc_data['YeoJohnson_Price_Back'] = (
    scaled_btc_data['YeoJohnson_Price'] * transformed_stds['Price_std']
) + transformed_means['Price_mean']

scaled_btc_data['YeoJohnson_Change_Back'] = (
    scaled_btc_data['YeoJohnson_Change'] * transformed_stds['Change_std']
) + transformed_means['Change_mean']

scaled_btc_data['YeoJohnson_7D_Pct_Change_Back'] = (
    scaled_btc_data['YeoJohnson_7D_Pct_Change'] * transformed_stds['Pct_Change_std_7D']
) + transformed_means['Pct_Change_mean_7D']

scaled_btc_data['YeoJohnson_2D_Pct_Change_Back'] = (
    scaled_btc_data['YeoJohnson_2D_Pct_Change'] * transformed_stds['Pct_Change_std_2D']
) + transformed_means['Pct_Change_mean_2D']

# Step 2: Create temporary DataFrames with the exact column names used during fit
price_back_df = scaled_btc_data[['YeoJohnson_Price_Back']].rename(columns={'YeoJohnson_Price_Back': 'Price'})
change_back_df = scaled_btc_data[['YeoJohnson_Change_Back']].rename(columns={'YeoJohnson_Change_Back': 'Change'})
pct_change_7d_back_df = scaled_btc_data[['YeoJohnson_7D_Pct_Change_Back']].rename(columns={'YeoJohnson_7D_Pct_Change_Back': 'Pct_Change_7D'})
pct_change_2d_back_df = scaled_btc_data[['YeoJohnson_2D_Pct_Change_Back']].rename(columns={'YeoJohnson_2D_Pct_Change_Back': 'Pct_Change_2D'})

# Step 3: Apply the inverse transformation to recover the original values
scaled_btc_data['Original_Price'] = power_transformer_price.inverse_transform(price_back_df)
scaled_btc_data['Original_Change'] = power_transformer_change.inverse_transform(change_back_df)
scaled_btc_data['Original_7D_Pct_Change'] = power_transformer_7d_pct_change.inverse_transform(pct_change_7d_back_df)
scaled_btc_data['Original_2D_Pct_Change'] = power_transformer_2d_pct_change.inverse_transform(pct_change_2d_back_df)

# Step 4: Display the back-transformed data
print("\nBack-Transformed Data:")
print(scaled_btc_data[['Original_Price', 'Original_Change', 'Original_7D_Pct_Change', 'Original_2D_Pct_Change']].head())

In [ ]:
# Recreate PowerTransformers with stored lambdas for back-transformation
power_transformer_price.lambdas_ = [lambdas['Price_lambda']]
power_transformer_change.lambdas_ = [lambdas['Change_lambda']]
power_transformer_7d_pct_change.lambdas_ = [lambdas['Pct_Change_lambda_7D']]
power_transformer_2d_pct_change.lambdas_ = [lambdas['Pct_Change_lambda_2D']]

# De-standardize the standardized values in final_df to get Yeo-Johnson transformed values
final_df['YeoJohnson_Price_Back'] = (
    final_df['YeoJohnson_Price'] * transformed_stds['Price_std']
) + transformed_means['Price_mean']

final_df['YeoJohnson_Change_Back'] = (
    final_df['YeoJohnson_Change'] * transformed_stds['Change_std']
) + transformed_means['Change_mean']

final_df['YeoJohnson_7D_Pct_Change_Back'] = (
    final_df['YeoJohnson_7D_Pct_Change'] * transformed_stds['Pct_Change_std_7D']
) + transformed_means['Pct_Change_mean_7D']

final_df['YeoJohnson_2D_Pct_Change_Back'] = (
    final_df['YeoJohnson_2D_Pct_Change'] * transformed_stds['Pct_Change_std_2D']
) + transformed_means['Pct_Change_mean_2D']

# Create temporary DataFrames with the exact column names used during fit
price_back_df = final_df[['YeoJohnson_Price_Back']].rename(columns={'YeoJohnson_Price_Back': 'Price'})
change_back_df = final_df[['YeoJohnson_Change_Back']].rename(columns={'YeoJohnson_Change_Back': 'Change'})
pct_change_7d_back_df = final_df[['YeoJohnson_7D_Pct_Change_Back']].rename(columns={'YeoJohnson_7D_Pct_Change_Back': 'Pct_Change_7D'})
pct_change_2d_back_df = final_df[['YeoJohnson_2D_Pct_Change_Back']].rename(columns={'YeoJohnson_2D_Pct_Change_Back': 'Pct_Change_2D'})

# Apply the inverse transformation to recover the original values
final_df['Original_Price'] = power_transformer_price.inverse_transform(price_back_df)
final_df['Original_Change'] = power_transformer_change.inverse_transform(change_back_df)
final_df['Original_7D_Pct_Change'] = power_transformer_7d_pct_change.inverse_transform(pct_change_7d_back_df)
final_df['Original_2D_Pct_Change'] = power_transformer_2d_pct_change.inverse_transform(pct_change_2d_back_df)

# Verify that the original data in final_df matches the original data in scaled_btc_data
if not all(final_df.index == scaled_btc_data.index):
    print("Warning: Indices don't match between final_df and scaled_btc_data")

    # Align the dataframes if needed
    common_dates = final_df.index.intersection(scaled_btc_data.index)
    print(f"Common dates count: {len(common_dates)}")

    # Create a new column in scaled_btc_data for verification
    scaled_btc_data_aligned = scaled_btc_data.loc[common_dates]
    final_df_aligned = final_df.loc[common_dates]

    # Verify values match (sample check)
    price_diff = (scaled_btc_data_aligned['Original_Price'] - final_df_aligned['Original_Price']).abs().mean()
    print(f"Average absolute difference in Original_Price between dataframes: {price_diff:.8f}")
else:
    print("Dataframe indices match perfectly")

    # Verify values match
    price_diff = (scaled_btc_data['Original_Price'] - final_df['Original_Price']).abs().mean()
    print(f"Average absolute difference in Original_Price between dataframes: {price_diff:.8f}")

# Create results DataFrame with predictions from each model
results = pd.DataFrame({
    'actual': y_test,  # actual test values
    'predicted': regressor.predict(X_test)  # regressor model (can change model here)
})

print(results[['actual', 'predicted']].head(10))
print(results['actual'].isna().sum(), "actual NaNs")
print(results['actual'].unique())

## Backtesting and Trading Strategy

In [ ]:
# Clone model
def clone_and_predict(model, X_train, y_train, X_pred):

    # Check if it's a statsmodels model
    if isinstance(model, sm.regression.linear_model.RegressionResultsWrapper):
        # For statsmodels models
        X_train_sm = sm.add_constant(X_train)
        X_pred_sm = sm.add_constant(X_pred)

        # Create a new model with the same formula
        new_model = sm.OLS(y_train, X_train_sm).fit()
        return new_model.predict(X_pred_sm)

    # For scikit-learn models
    elif hasattr(model, 'fit') and hasattr(model, 'predict'):
        # Create a clone
        if hasattr(model, 'get_params'):
            model_clone = type(model)(**model.get_params())
        else:
            model_clone = deepcopy(model)

        # Fit and predict
        model_clone.fit(X_train, y_train)
        return model_clone.predict(X_pred)

    else:
        raise TypeError("Unsupported model type. Model must be either a statsmodels result or have fit/predict methods.")

In [ ]:
# Generate predictions using walk-forward validation

def generate_predictions(model, X, target_col, test_size=0.3, step_size=0.05):
    """
    Parameters:
    model : model with fit and predict methods or statsmodels result
    X : DataFrame with features and target
    target_col : target column name
    test_size : fraction of data to use for testing
    step_size : fraction of data to move forward in each step
    """
    # Extract features and target
    feature_cols = [col for col in X.columns if not any(x in col for x in ['YeoJohnson_', 'Original_'])]
    X_features = X[feature_cols].copy()
    target_series = X[target_col].copy()

    # Calculate indices
    data_size = len(X_features)
    test_start_idx = int(data_size * (1 - test_size))
    step_idx = max(1, int(data_size * step_size))

    # Create results DataFrame
    results = pd.DataFrame(index=X_features.index[test_start_idx:])
    results['actual'] = target_series.iloc[test_start_idx:].values
    results['predicted'] = np.nan

    # Add price if available
    if 'Original_Price' in X.columns:
        results['price'] = X['Original_Price'].iloc[test_start_idx:].values

    # Walk-forward testing
    train_end_idx = test_start_idx
    with tqdm(total=data_size - test_start_idx) as pbar:
        while train_end_idx < data_size:
            # Define train set and prediction window
            X_train = X_features.iloc[:train_end_idx]
            y_train = target_series.iloc[:train_end_idx]
            pred_end_idx = min(train_end_idx + step_idx, data_size)
            X_pred = X_features.iloc[train_end_idx:pred_end_idx]

            if len(X_pred) == 0:
                break

            # Handle different model types for prediction
            predictions = clone_and_predict(model, X_train, y_train, X_pred)

            # Store predictions
            pred_indices = X_features.index[train_end_idx:pred_end_idx]
            results.loc[pred_indices, 'predicted'] = predictions

            # Move forward
            train_end_idx = pred_end_idx
            pbar.update(min(step_idx, pred_end_idx - (train_end_idx - step_idx)))

    # Calculate metrics
    mse = mean_squared_error(results['actual'].dropna(), results['predicted'].dropna())
    rmse = np.sqrt(mse)
    r2 = r2_score(results['actual'].dropna(), results['predicted'].dropna())

    print(f"Prediction Metrics: MSE={mse:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")

    return results

In [ ]:
# More Conservative Trading Strategy
def apply_trading_strategy(prediction_results, target_col,
                          initial_capital=10000, position_size=0.95,
                          stop_loss=None, take_profit=None):
    """Apply a more conservative trading strategy to predictions"""
    results = prediction_results.copy()

    # Calculate prediction reliability metrics
    results['pred_error'] = results['actual'] - results['predicted']
    results['rolling_error'] = results['pred_error'].rolling(window=10, min_periods=1).mean()
    results['error_std'] = results['pred_error'].rolling(window=10, min_periods=1).std()

    # Create improved signal with stricter criteria and more conservatism
    # Only take positions when predicted value exceeds a threshold (not just > 0)
    threshold = 4.0  # Only trade when predicted change is significant (3%)
    confidence_margin = 1.0  # Additional margin to reduce false signals

    # Create directional signal (1 for strong buy, -1 for strong sell, 0 otherwise)
    results['raw_signal'] = np.where(results['predicted'] > threshold + confidence_margin, 1,
                           np.where(results['predicted'] < -threshold - confidence_margin, -1, 0))

    # Add trend filter - only trade when prediction aligns with recent trend
    results['price_ma5'] = results['price'].rolling(window=5, min_periods=1).mean() if 'price' in results.columns else 0
    results['price_ma20'] = results['price'].rolling(window=20, min_periods=1).mean() if 'price' in results.columns else 0

    # Trade only if price is above 20-day MA for buys or below for sells (trend confirmation)
    if 'price' in results.columns:
        trend_aligned = ((results['raw_signal'] == 1) & (results['price'] > results['price_ma20'])) | \
                        ((results['raw_signal'] == -1) & (results['price'] < results['price_ma20'])) | \
                        (results['raw_signal'] == 0)
        results['signal'] = np.where(trend_aligned, results['raw_signal'], 0)
    else:
        results['signal'] = results['raw_signal']

    # Filter out signals when error is high (model is less reliable)
    error_threshold = results['error_std'].mean() * 2  # Dynamic threshold based on typical error
    results['signal'] = np.where(results['error_std'] > error_threshold, 0, results['signal'])

    # Add position sizing based on conviction (stronger signals get larger positions)
    results['conviction'] = np.abs(results['predicted']) / (results['error_std'] + 0.1)  # Prevent div by zero
    results['conviction'] = results['conviction'] / results['conviction'].rolling(window=30, min_periods=1).max().fillna(1)
    results['position_pct'] = results['signal'] * np.minimum(results['conviction'] * 0.8, 1.0) * position_size

    # Shift signal to avoid look-ahead bias
    results['signal'] = results['signal'].shift(1).fillna(0)
    results['position_pct'] = results['position_pct'].shift(1).fillna(0)

    # Calculate actual returns based on target type
    if '7D_Pct_Change' in target_col or '2D_Pct_Change' in target_col:
        results['actual_return'] = results['actual'] / 100  # Convert % to decimal
    else:
        # For price or change targets
        if 'price' in results.columns:
            results['actual_return'] = results['price'].pct_change()
        else:
            results['actual_return'] = results['actual']

    # Portfolio simulation
    results['capital'] = initial_capital
    results['position'] = 0
    results['shares'] = 0
    results['cash'] = initial_capital

    # Set up for price tracking
    if 'price' in results.columns:
        price_col = 'price'
    else:
        # Create synthetic price starting at 100
        results['synth_price'] = 100
        for i in range(1, len(results)):
            results['synth_price'].iloc[i] = results['synth_price'].iloc[i-1] * (1 + results['actual_return'].iloc[i])
        price_col = 'synth_price'

    # Add volatility-based position sizing and dynamic stops
    if 'price' in results.columns:
        # Calculate volatility metric (ATR-like)
        results['daily_range'] = results['price'].rolling(window=14).std()
        results['volatility_factor'] = 1.0 - np.minimum(results['daily_range'] / results['price'] * 20, 0.5)
    else:
        results['volatility_factor'] = 0.8  # Default conservative factor

    # Implement trading logic
    entry_price = 0
    current_position = 0
    days_in_position = 0
    max_position_days = 15  # Maximum days to hold a position

    # Conservative money management
    max_trades_open = 3  # Maximum number of trades to have open at once
    trades_today = 0  # Count of trades opened today
    max_daily_trades = 1  # Maximum trades to open per day
    open_trades = 0  # Number of currently open trades

    for i in range(1, len(results)):
        prev_idx = results.index[i-1]
        curr_idx = results.index[i]

        # Get values from previous day
        prev_signal = results.loc[prev_idx, 'signal']
        prev_position_pct = results.loc[prev_idx, 'position_pct']
        prev_cash = results.loc[prev_idx, 'cash']
        prev_shares = results.loc[prev_idx, 'shares']
        volatility_factor = results.loc[prev_idx, 'volatility_factor']

        # Current price
        curr_price = results.loc[curr_idx, price_col]

        # Increment days in position if we have one
        if current_position != 0:
            days_in_position += 1

        # Force close if in position too long (prevents holding losing trades)
        if days_in_position > max_position_days and current_position != 0:
            if current_position == 1:  # Close long
                cash_value = prev_shares * curr_price
                prev_cash = cash_value
                prev_shares = 0
            elif current_position == -1:  # Close short
                cash_value = prev_cash + (entry_price - curr_price) * abs(prev_shares)
                prev_cash = cash_value
                prev_shares = 0

            current_position = 0
            entry_price = 0
            days_in_position = 0
            open_trades -= 1
            continue

        # Dynamic stop loss/take profit based on volatility
        if stop_loss is not None:
            dynamic_stop = stop_loss * (1 / volatility_factor)  # Tighter stops in high volatility
        else:
            dynamic_stop = 0.03  # Default 3% stop if none specified

        if take_profit is not None:
            dynamic_tp = take_profit * volatility_factor  # Adjust TP based on volatility
        else:
            dynamic_tp = 0.06  # Default 6% take profit

        # Check stop-loss/take-profit
        if current_position != 0 and entry_price > 0:
            if current_position == 1:  # Long position
                current_return = (curr_price / entry_price) - 1
                # Stop loss
                if current_return <= -dynamic_stop:
                    cash_value = prev_shares * curr_price
                    current_position = 0
                    entry_price = 0
                    prev_signal = 0
                    prev_cash = cash_value
                    prev_shares = 0
                    days_in_position = 0
                    open_trades -= 1
                # Take profit
                elif current_return >= dynamic_tp:
                    cash_value = prev_shares * curr_price
                    current_position = 0
                    entry_price = 0
                    prev_signal = 0
                    prev_cash = cash_value
                    prev_shares = 0
                    days_in_position = 0
                    open_trades -= 1

            elif current_position == -1:  # Short position
                current_return = 1 - (curr_price / entry_price)
                # Stop loss
                if current_return <= -dynamic_stop:
                    cash_value = prev_cash + (entry_price - curr_price) * abs(prev_shares)
                    current_position = 0
                    entry_price = 0
                    prev_signal = 0
                    prev_cash = cash_value
                    prev_shares = 0
                    days_in_position = 0
                    open_trades -= 1
                # Take profit
                elif current_return >= dynamic_tp:
                    cash_value = prev_cash + (entry_price - curr_price) * abs(prev_shares)
                    current_position = 0
                    entry_price = 0
                    prev_signal = 0
                    prev_cash = cash_value
                    prev_shares = 0
                    days_in_position = 0
                    open_trades -= 1

        # Reset daily trade counter if it's a new day
        if i > 1 and results.index[i].date() != results.index[i-1].date():
            trades_today = 0

        # Process trading signal if different from current position
        if prev_signal != current_position and trades_today < max_daily_trades and open_trades < max_trades_open:
            # Close existing position
            if current_position == 1:  # Close long
                cash_value = prev_shares * curr_price
                prev_cash = cash_value
                prev_shares = 0
                current_position = 0
                entry_price = 0
                days_in_position = 0
                open_trades -= 1
            elif current_position == -1:  # Close short
                cash_value = prev_cash + (entry_price - curr_price) * abs(prev_shares)
                prev_cash = cash_value
                prev_shares = 0
                current_position = 0
                entry_price = 0
                days_in_position = 0
                open_trades -= 1

            # Open new position with adjusted position size
            if prev_signal != 0:
                # Adjust position size based on volatility factor and conviction
                adjusted_position_size = abs(prev_position_pct) * volatility_factor
                position_value = prev_cash * adjusted_position_size

                if position_value > 0:  # Only trade if position value is positive
                    if prev_signal == 1:  # Long
                        prev_shares = position_value / curr_price
                        prev_cash -= position_value
                        current_position = 1
                        entry_price = curr_price
                        days_in_position = 0
                        trades_today += 1
                        open_trades += 1
                    elif prev_signal == -1:  # Short
                        prev_shares = -position_value / curr_price
                        current_position = -1
                        entry_price = curr_price
                        days_in_position = 0
                        trades_today += 1
                        open_trades += 1

        # Calculate current capital
        if current_position == 1:  # Long
            curr_capital = prev_cash + (prev_shares * curr_price)
        elif current_position == -1:  # Short
            short_pnl = (entry_price - curr_price) * abs(prev_shares)
            curr_capital = prev_cash + short_pnl
        else:  # No position
            curr_capital = prev_cash

        # Update results
        results.loc[curr_idx, 'capital'] = curr_capital
        results.loc[curr_idx, 'position'] = current_position
        results.loc[curr_idx, 'shares'] = prev_shares
        results.loc[curr_idx, 'cash'] = prev_cash

    # Calculate performance metrics
    results['daily_return'] = results['capital'].pct_change()
    results['cum_return'] = (1 + results['daily_return']).cumprod() - 1

    # Market returns (buy & hold)
    results['market_return'] = results[price_col].pct_change()
    results['cum_market_return'] = (1 + results['market_return']).cumprod() - 1

    # Calculate drawdowns
    results['peak_capital'] = results['capital'].cummax()
    results['drawdown'] = (results['capital'] - results['peak_capital']) / results['peak_capital']

    return results

In [ ]:
# Backtesting - Performance Analysis
def analyze_backtest(backtest_results):
    """Calculate performance metrics and display results"""
    results = backtest_results.copy()

    # Calculate basic metrics
    total_return = results['cum_return'].iloc[-1]
    market_return = results['cum_market_return'].iloc[-1]
    max_drawdown = results['drawdown'].min()

    # Calculate annualized return
    days = (results.index[-1] - results.index[0]).days
    years = max(days / 365, 0.01)
    annual_return = (1 + total_return) ** (1/years) - 1
    market_annual = (1 + market_return) ** (1/years) - 1

    # Calculate Sharpe ratio (annualized)
    risk_free_rate = 0.02  # 2% annual risk-free rate
    excess_returns = results['daily_return'] - ((1 + risk_free_rate) ** (1/252) - 1)
    sharpe_ratio = np.sqrt(252) * excess_returns.mean() / excess_returns.std() if excess_returns.std() > 0 else 0

    # Calculate win rate and trades
    trade_changes = results['position'].diff().fillna(0)
    total_trades = (trade_changes != 0).sum() / 2
    winning_days = (results['daily_return'] > 0).sum()
    losing_days = (results['daily_return'] < 0).sum()
    win_rate = winning_days / (winning_days + losing_days) if (winning_days + losing_days) > 0 else 0

    # Print results
    print("\nPerformance Summary:")
    print(f"Total Return: {total_return:.2%} (Market: {market_return:.2%})")
    print(f"Annual Return: {annual_return:.2%} (Market: {market_annual:.2%})")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Max Drawdown: {max_drawdown:.2%}")
    print(f"Total Trades: {total_trades:.0f}")
    print(f"Win Rate: {win_rate:.2%} ({winning_days}/{winning_days+losing_days})")

    return {
        'total_return': total_return,
        'annual_return': annual_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'total_trades': total_trades,
        'win_rate': win_rate
    }

In [ ]:
# Visualization
def visualize_backtest(backtest_results, target_col):
    """Create visualization of backtest results"""
    results = backtest_results.copy()

    # Plot 1: Actual vs Predicted
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['actual'], 'b-', label='Actual')
    plt.plot(results.index, results['predicted'], 'r-', label='Predicted')

    if '7D_Pct_Change' in target_col:
        title = '7-Day Percentage Change'
        ylabel = '7-Day % Change'
    elif '2D_Pct_Change' in target_col:
        title = '2-Day Percentage Change'
        ylabel = '2-Day % Change'
    else:
        title = 'Value'
        ylabel = 'Value'

    plt.title(f'Actual vs Predicted {title}')
    plt.xlabel('Date')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot 2: Capital Growth
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['capital'], 'g-', label='Strategy Capital')

    # Add buy-and-hold for comparison
    price_col = 'price' if 'price' in results.columns else 'synth_price'
    initial_capital = results['capital'].iloc[0]
    initial_price = results[price_col].iloc[0]
    initial_shares = initial_capital / initial_price
    buy_hold_capital = initial_shares * results[price_col]
    plt.plot(results.index, buy_hold_capital, 'b-', label='Buy & Hold')

    plt.title('Capital Growth')
    plt.xlabel('Date')
    plt.ylabel('Capital ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot 3: Cumulative Returns
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['cum_return'], 'g-', label='Strategy')
    plt.plot(results.index, results['cum_market_return'], 'b-', label='Market')
    plt.title('Cumulative Returns')
    plt.xlabel('Date')
    plt.ylabel('Return')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot 4: Drawdowns
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['drawdown'], 'r-')
    plt.title('Strategy Drawdowns')
    plt.xlabel('Date')
    plt.ylabel('Drawdown')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot 5: Position over time
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['position'], 'b-')
    plt.title('Position Over Time')
    plt.xlabel('Date')
    plt.ylabel('Position (1=Long, -1=Short, 0=Cash)')
    plt.yticks([-1, 0, 1])
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
def fixed_backtest_model(model, X, target_col, test_size=0.25, step_size=0.02,
                       initial_capital=10000, position_size=1.0,
                       stop_loss=None, take_profit=None, plot=False):
    """
    A fixed version that handles the formula model correctly
    """
    # Check if it's a formula-based model
    is_formula_model = isinstance(model, sm.regression.linear_model.RegressionResultsWrapper) and hasattr(model.model, 'formula')

    if is_formula_model:
        print("Using formula-based model...")
        # Get the formula
        formula = model.model.formula
        print(f"Model formula: {formula}")

        # Step 1: Generate predictions using the formula directly
        print("Step 1: Generating predictions...")

        # Simplified approach - recreate model for each prediction
        # Instead of using walk-forward validation, we'll just split the data
        # This is less accurate but easier to implement
        data_size = len(X)
        test_start_idx = int(data_size * (1 - test_size))

        # Train data
        train_data = X.iloc[:test_start_idx].copy()
        # Test data
        test_data = X.iloc[test_start_idx:].copy()

        # Re-fit model on training data
        import statsmodels.formula.api as smf
        train_model = smf.ols(formula=formula, data=train_data).fit()

        # Make predictions
        test_predictions = train_model.predict(test_data)

        # Create prediction results dataframe
        prediction_results = pd.DataFrame(index=test_data.index)
        prediction_results['actual'] = test_data[target_col].values
        prediction_results['predicted'] = test_predictions

        if 'Original_Price' in X.columns:
            prediction_results['price'] = X['Original_Price'].iloc[test_start_idx:].values
    else:
        # For non-formula models, use your original approach
        print("Using standard model...")
        prediction_results = generate_predictions(
            model=model, X=X, target_col=target_col,
            test_size=test_size, step_size=step_size
        )

    # Continue with trading strategy as before
    print("\nStep 2: Applying trading strategy...")
    backtest_results = apply_trading_strategy(
        prediction_results=prediction_results,
        target_col=target_col,
        initial_capital=initial_capital,
        position_size=position_size,
        stop_loss=stop_loss,
        take_profit=take_profit
    )

    # Analysis and visualization remain the same
    print("\nStep 3: Analyzing performance...")
    performance = analyze_backtest(backtest_results)

    if plot:
        print("\nStep 4: Generating visualizations...")
        visualize_backtest(backtest_results, target_col)

    return backtest_results, performance

In [ ]:
# Create backtest df
backtest_df = final_df.copy()

# Run backtest with clean DataFrame
backtest_results_basic, performance_basic = fixed_backtest_model(
    model=model,
    X=backtest_df,
    target_col='Original_7D_Pct_Change',
    test_size=0.25,
    step_size=0.02,
    initial_capital=25000,
    position_size=0.8,
    stop_loss=0.02,
    take_profit=0.08,
    plot=True
)

backup (del later)

In [ ]:
# Clone model
def clone_and_predict(model, X_train, y_train, X_pred):

    # Check if it's a statsmodels model
    if isinstance(model, sm.regression.linear_model.RegressionResultsWrapper):
        # For statsmodels models
        X_train_sm = sm.add_constant(X_train)
        X_pred_sm = sm.add_constant(X_pred)

        # Create a new model with the same formula
        new_model = sm.OLS(y_train, X_train_sm).fit()
        return new_model.predict(X_pred_sm)

    # For scikit-learn models
    elif hasattr(model, 'fit') and hasattr(model, 'predict'):
        # Create a clone
        if hasattr(model, 'get_params'):
            model_clone = type(model)(**model.get_params())
        else:
            model_clone = deepcopy(model)

        # Fit and predict
        model_clone.fit(X_train, y_train)
        return model_clone.predict(X_pred)

    else:
        raise TypeError("Unsupported model type. Model must be either a statsmodels result or have fit/predict methods.")

In [ ]:
# Generate predictions using walk-forward validation

def generate_predictions(model, X, target_col, test_size=0.3, step_size=0.05):
    """
    Parameters:
    model : model with fit and predict methods or statsmodels result
    X : DataFrame with features and target
    target_col : target column name
    test_size : fraction of data to use for testing
    step_size : fraction of data to move forward in each step
    """
    # Extract features and target
    feature_cols = [col for col in X.columns if not any(x in col for x in ['YeoJohnson_', 'Original_'])]
    X_features = X[feature_cols].copy()
    target_series = X[target_col].copy()

    # Calculate indices
    data_size = len(X_features)
    test_start_idx = int(data_size * (1 - test_size))
    step_idx = max(1, int(data_size * step_size))

    # Create results DataFrame
    results = pd.DataFrame(index=X_features.index[test_start_idx:])
    results['actual'] = target_series.iloc[test_start_idx:].values
    results['predicted'] = np.nan

    # Add price if available
    if 'Original_Price' in X.columns:
        results['price'] = X['Original_Price'].iloc[test_start_idx:].values

    # Walk-forward testing
    train_end_idx = test_start_idx
    with tqdm(total=data_size - test_start_idx) as pbar:
        while train_end_idx < data_size:
            # Define train set and prediction window
            X_train = X_features.iloc[:train_end_idx]
            y_train = target_series.iloc[:train_end_idx]
            pred_end_idx = min(train_end_idx + step_idx, data_size)
            X_pred = X_features.iloc[train_end_idx:pred_end_idx]

            if len(X_pred) == 0:
                break

            # Handle different model types for prediction
            predictions = clone_and_predict(model, X_train, y_train, X_pred)

            # Store predictions
            pred_indices = X_features.index[train_end_idx:pred_end_idx]
            results.loc[pred_indices, 'predicted'] = predictions

            # Move forward
            train_end_idx = pred_end_idx
            pbar.update(min(step_idx, pred_end_idx - (train_end_idx - step_idx)))

    # Calculate metrics
    mse = mean_squared_error(results['actual'].dropna(), results['predicted'].dropna())
    rmse = np.sqrt(mse)
    r2 = r2_score(results['actual'].dropna(), results['predicted'].dropna())

    print(f"Prediction Metrics: MSE={mse:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")

    return results

In [ ]:
# Trading Strategy
def apply_trading_strategy(prediction_results, target_col,
                          initial_capital=10000, position_size=0.95,
                          stop_loss=None, take_profit=None):
    """Apply trading strategy to predictions"""
    results = prediction_results.copy()

    # Create signal: 1 for buy (predicted > 0), -1 for sell (predicted < 0)
    results['signal'] = np.sign(results['predicted'])

    # Calculate actual returns based on target type
    if '7D_Pct_Change' in target_col or '2D_Pct_Change' in target_col:
        results['actual_return'] = results['actual'] / 100  # Convert % to decimal
    else:
        # For price or change targets
        if 'price' in results.columns:
            results['actual_return'] = results['price'].pct_change()
        else:
            results['actual_return'] = results['actual']

    # Shift signal to avoid look-ahead bias
    results['signal'] = results['signal'].shift(1)
    results['signal'].iloc[0] = 0  # No position on first day

    # Portfolio simulation
    results['capital'] = initial_capital
    results['position'] = 0
    results['shares'] = 0
    results['cash'] = initial_capital

    # Set up for price tracking
    if 'price' in results.columns:
        price_col = 'price'
    else:
        # Create synthetic price starting at 100
        results['synth_price'] = 100
        for i in range(1, len(results)):
            results['synth_price'].iloc[i] = results['synth_price'].iloc[i-1] * (1 + results['actual_return'].iloc[i])
        price_col = 'synth_price'

    # Implement trading logic
    entry_price = 0
    current_position = 0

    for i in range(1, len(results)):
        prev_idx = results.index[i-1]
        curr_idx = results.index[i]

        # Get values from previous day
        prev_signal = results.loc[prev_idx, 'signal']
        prev_cash = results.loc[prev_idx, 'cash']
        prev_shares = results.loc[prev_idx, 'shares']

        # Current price
        curr_price = results.loc[curr_idx, price_col]

        # Check stop-loss/take-profit
        if current_position != 0 and entry_price > 0:
            if current_position == 1:  # Long position
                current_return = (curr_price / entry_price) - 1
                # Stop loss
                if stop_loss is not None and current_return <= -stop_loss:
                    cash_value = prev_shares * curr_price
                    current_position = 0
                    entry_price = 0
                    prev_signal = 0
                    prev_cash = cash_value
                    prev_shares = 0
                # Take profit
                elif take_profit is not None and current_return >= take_profit:
                    cash_value = prev_shares * curr_price
                    current_position = 0
                    entry_price = 0
                    prev_signal = 0
                    prev_cash = cash_value
                    prev_shares = 0

            elif current_position == -1:  # Short position
                current_return = 1 - (curr_price / entry_price)
                # Stop loss
                if stop_loss is not None and current_return <= -stop_loss:
                    cash_value = prev_cash + (entry_price - curr_price) * abs(prev_shares)
                    current_position = 0
                    entry_price = 0
                    prev_signal = 0
                    prev_cash = cash_value
                    prev_shares = 0
                # Take profit
                elif take_profit is not None and current_return >= take_profit:
                    cash_value = prev_cash + (entry_price - curr_price) * abs(prev_shares)
                    current_position = 0
                    entry_price = 0
                    prev_signal = 0
                    prev_cash = cash_value
                    prev_shares = 0

        # Process trading signal if different from current position
        if prev_signal != current_position:
            # Close existing position
            if current_position == 1:  # Close long
                cash_value = prev_shares * curr_price
                prev_cash = cash_value
                prev_shares = 0
            elif current_position == -1:  # Close short
                cash_value = prev_cash + (entry_price - curr_price) * abs(prev_shares)
                prev_cash = cash_value
                prev_shares = 0

            # Open new position
            if prev_signal != 0:
                position_value = prev_cash * position_size

                if prev_signal == 1:  # Long
                    prev_shares = position_value / curr_price
                    prev_cash -= position_value
                    current_position = 1
                    entry_price = curr_price
                elif prev_signal == -1:  # Short
                    prev_shares = -position_value / curr_price
                    current_position = -1
                    entry_price = curr_price

        # Calculate current capital
        if current_position == 1:  # Long
            curr_capital = prev_cash + (prev_shares * curr_price)
        elif current_position == -1:  # Short
            short_pnl = (entry_price - curr_price) * abs(prev_shares)
            curr_capital = prev_cash + short_pnl
        else:  # No position
            curr_capital = prev_cash

        # Update results
        results.loc[curr_idx, 'capital'] = curr_capital
        results.loc[curr_idx, 'position'] = current_position
        results.loc[curr_idx, 'shares'] = prev_shares
        results.loc[curr_idx, 'cash'] = prev_cash

    # Calculate performance metrics
    results['daily_return'] = results['capital'].pct_change()
    results['cum_return'] = (1 + results['daily_return']).cumprod() - 1

    # Market returns (buy & hold)
    results['market_return'] = results[price_col].pct_change()
    results['cum_market_return'] = (1 + results['market_return']).cumprod() - 1

    # Calculate drawdowns
    results['peak_capital'] = results['capital'].cummax()
    results['drawdown'] = (results['capital'] - results['peak_capital']) / results['peak_capital']

    return results

In [ ]:
# Backtesting - Performance Analysis
def analyze_backtest(backtest_results):
    """Calculate performance metrics and display results"""
    results = backtest_results.copy()

    # Calculate basic metrics
    total_return = results['cum_return'].iloc[-1]
    market_return = results['cum_market_return'].iloc[-1]
    max_drawdown = results['drawdown'].min()

    # Calculate annualized return
    days = (results.index[-1] - results.index[0]).days
    years = max(days / 365, 0.01)
    annual_return = (1 + total_return) ** (1/years) - 1
    market_annual = (1 + market_return) ** (1/years) - 1

    # Calculate Sharpe ratio (annualized)
    risk_free_rate = 0.02  # 2% annual risk-free rate
    excess_returns = results['daily_return'] - ((1 + risk_free_rate) ** (1/252) - 1)
    sharpe_ratio = np.sqrt(252) * excess_returns.mean() / excess_returns.std() if excess_returns.std() > 0 else 0

    # Calculate win rate and trades
    trade_changes = results['position'].diff().fillna(0)
    total_trades = (trade_changes != 0).sum() / 2
    winning_days = (results['daily_return'] > 0).sum()
    losing_days = (results['daily_return'] < 0).sum()
    win_rate = winning_days / (winning_days + losing_days) if (winning_days + losing_days) > 0 else 0

    # Print results
    print("\nPerformance Summary:")
    print(f"Total Return: {total_return:.2%} (Market: {market_return:.2%})")
    print(f"Annual Return: {annual_return:.2%} (Market: {market_annual:.2%})")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Max Drawdown: {max_drawdown:.2%}")
    print(f"Total Trades: {total_trades:.0f}")
    print(f"Win Rate: {win_rate:.2%} ({winning_days}/{winning_days+losing_days})")

    return {
        'total_return': total_return,
        'annual_return': annual_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'total_trades': total_trades,
        'win_rate': win_rate
    }

In [ ]:
# Visualization
def visualize_backtest(backtest_results, target_col):
    """Create visualization of backtest results"""
    results = backtest_results.copy()

    # Plot 1: Actual vs Predicted
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['actual'], 'b-', label='Actual')
    plt.plot(results.index, results['predicted'], 'r-', label='Predicted')

    if '7D_Pct_Change' in target_col:
        title = '7-Day Percentage Change'
        ylabel = '7-Day % Change'
    elif '2D_Pct_Change' in target_col:
        title = '2-Day Percentage Change'
        ylabel = '2-Day % Change'
    else:
        title = 'Value'
        ylabel = 'Value'

    plt.title(f'Actual vs Predicted {title}')
    plt.xlabel('Date')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot 2: Capital Growth
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['capital'], 'g-', label='Strategy Capital')

    # Add buy-and-hold for comparison
    price_col = 'price' if 'price' in results.columns else 'synth_price'
    initial_capital = results['capital'].iloc[0]
    initial_price = results[price_col].iloc[0]
    initial_shares = initial_capital / initial_price
    buy_hold_capital = initial_shares * results[price_col]
    plt.plot(results.index, buy_hold_capital, 'b-', label='Buy & Hold')

    plt.title('Capital Growth')
    plt.xlabel('Date')
    plt.ylabel('Capital ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot 3: Cumulative Returns
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['cum_return'], 'g-', label='Strategy')
    plt.plot(results.index, results['cum_market_return'], 'b-', label='Market')
    plt.title('Cumulative Returns')
    plt.xlabel('Date')
    plt.ylabel('Return')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot 4: Drawdowns
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['drawdown'], 'r-')
    plt.title('Strategy Drawdowns')
    plt.xlabel('Date')
    plt.ylabel('Drawdown')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot 5: Position over time
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['position'], 'b-')
    plt.title('Position Over Time')
    plt.xlabel('Date')
    plt.ylabel('Position (1=Long, -1=Short, 0=Cash)')
    plt.yticks([-1, 0, 1])
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Overall Model Prediction
def backtest_model(model, X, target_col='Original_7D_Pct_Change',
                  test_size=0.3, step_size=0.05,
                  initial_capital=10000, position_size=0.95,
                  stop_loss=None, take_profit=None,
                  plot=True):
    """Main function to run backtest with model predictions"""

    # Step 1: Generate predictions
    print("Step 1: Generating predictions...")
    prediction_results = generate_predictions(
        model=model, X=X, target_col=target_col,
        test_size=test_size, step_size=step_size
    )

    # Step 2: Apply trading strategy
    print("\nStep 2: Applying trading strategy...")
    backtest_results = apply_trading_strategy(
        prediction_results=prediction_results,
        target_col=target_col,
        initial_capital=initial_capital,
        position_size=position_size,
        stop_loss=stop_loss,
        take_profit=take_profit
    )

    # Step 3: Analyze results
    print("\nStep 3: Analyzing performance...")
    performance = analyze_backtest(backtest_results)

    # Step 4: Visualize results
    if plot:
        print("\nStep 4: Generating visualizations...")
        visualize_backtest(backtest_results, target_col)

    return backtest_results, performance

In [ ]:
feature_cols = [col for col in final_df.columns if not any(x in col for x in [
    'YeoJohnson_', 'Original_']+ exclude_list)]

# Add the necessary columns for backtesting
backtest_df = final_df[feature_cols].copy()
backtest_df['Original_7D_Pct_Change'] = final_df['Original_7D_Pct_Change']  # target
backtest_df['Original_Price'] = final_df['Original_Price']  # for calculating returns

# Run backtest with clean DataFrame
backtest_results_basic, performance_basic = backtest_model(
    model=regressor,
    X=backtest_df,
    target_col='Original_7D_Pct_Change',
    test_size=0.25,
    step_size=0.02,
    initial_capital=25000,
    position_size=0.8,
    stop_loss=0.04,
    take_profit=0.08,
    plot=True
)


'''# Example 1: Random Forest model with default settings
backtest_results_rf, performance_rf = backtest_model(
    model=rf_tuned_regressor,
    X=backtest_df,
    target_col='Original_7D_Pct_Change'
)'''

'''# Example 2: Baseline OLS model with modified test size and step size
backtest_results_baseline, performance_baseline = backtest_model(
    model=model, # baseline OLS model
    X=backtest_df,
    target_col='Original_7D_Pct_Change',
    test_size=0.2,  # Use only 20% of data for testing
    step_size=0.1,   # Take larger steps in walk-forward validation
    initial_capital=10000,
    position_size=0.95,
    stop_loss=0.05,
    take_profit=0.05,)'''

'''# Example 3: Reduced model with different capital and position size
backtest_results_reduced, performance_reduced = backtest_model(
    model=model_reduce,
    X=backtest_df,
    target_col='Original_7D_Pct_Change',
    initial_capital=50000,  # Start with $50,000
    position_size=0.75      # Use 75% of capital per trade
)

# Example 4: VIF-reduced model with stop loss and take profit
backtest_results_vif, performance_vif = backtest_model(
    model=model_reduce_vif,
    X=backtest_df,
    target_col='Original_7D_Pct_Change',
    stop_loss=0.03,     # 3% stop loss
    take_profit=0.05    # 5% take profit
)

# Example 5: Final RF model with custom settings
backtest_results_final_rf, performance_final_rf = backtest_model(
    model=final_rfmodel,
    X=backtest_df,
    target_col='Original_7D_Pct_Change',
    test_size=0.25,
    step_size=0.02,
    initial_capital=25000,
    position_size=0.8,
    stop_loss=0.04,
    take_profit=0.08,
    plot=True
)

# Example 6: Basic regressor
backtest_results_basic, performance_basic = backtest_model(
    model=regressor,
    X=scaled_btc_data,
    target_col='Original_7D_Pct_Change',
    test_size=0.25,
    step_size=0.02,
    initial_capital=25000,
    position_size=0.8,
    stop_loss=0.04,
    take_profit=0.08,
    plot=True
)

# Example 7: Testing different target column with RF model
backtest_results_2d, performance_2d = backtest_model(
    model=rf_tuned_regressor,
    X=scaled_btc_data,
    target_col='Original_2D_Pct_Change',  # Predict 2-day changes instead
    test_size=0.3,
    step_size=0.05
)'''

##Predicting % of Bitcoin Price Change in 7 Days

In [ ]:
def predict_future_bitcoin_change(model, current_data, confidence_level=0.95):
    """
    Predict the 7-day percentage change in Bitcoin price from today.

    Parameters:
    -----------
    model : trained model
        The trained prediction model
    current_data : DataFrame
        Contains the latest feature values
    confidence_level : float
        Confidence level for prediction intervals
    """
    # Extract only features the model was trained with
    if hasattr(model, 'feature_names_in_'):
        model_features = list(model.feature_names_in_)
    else:
        model_features = [col for col in current_data.columns
                          if not any(x in col for x in ['YeoJohnson_', 'Original_'])]

    # Get current date and price
    current_date = current_data.index[-1]
    current_price = current_data['Original_Price'].iloc[-1] if 'Original_Price' in current_data.columns else None
    target_date = current_date + pd.Timedelta(days=7)

    # Make prediction
    latest_features = current_data[model_features].iloc[-1:].copy()
    prediction = model.predict(latest_features)[0]

    # Calculate confidence interval (for RandomForest models)
    if hasattr(model, 'estimators_'):
        tree_predictions = [tree.predict(latest_features)[0] for tree in model.estimators_]
        alpha = 1 - confidence_level
        lower_bound = np.percentile(tree_predictions, (alpha/2)*100)
        upper_bound = np.percentile(tree_predictions, (1-alpha/2)*100)
        std_dev = np.std(tree_predictions)
    else:
        std_dev = abs(prediction) * 0.2
        lower_bound = prediction - 1.96 * std_dev
        upper_bound = prediction + 1.96 * std_dev

    # Determine signal
    if prediction > 2.0:
        signal = "BUY"
    elif prediction > 0:
        signal = "WEAK BUY"
    elif prediction < -2.0:
        signal = "SELL"
    elif prediction < 0:
        signal = "WEAK SELL"
    else:
        signal = "NEUTRAL"

    # Calculate price targets if price data is available
    if current_price:
        predicted_price = current_price * (1 + prediction/100)
        lower_price = current_price * (1 + lower_bound/100)
        upper_price = current_price * (1 + upper_bound/100)

        # Calculate stop loss and take profit
        if signal in ["BUY", "WEAK BUY"]:
            stop_loss = current_price * 0.95
            take_profit = current_price * (1 + max(prediction/100 * 1.5, 0.1))
        elif signal in ["SELL", "WEAK SELL"]:
            stop_loss = current_price * 1.05
            take_profit = current_price * (1 + min(prediction/100 * 1.5, -0.1))
        else:
            stop_loss = take_profit = None

    # Display results
    print(f"\n{'='*60}")
    print(f"BITCOIN 7-DAY PRICE PREDICTION ({confidence_level*100:.0f}% CONFIDENCE)")
    print(f"{'='*60}")
    print(f"Current Date: {current_date.strftime('%Y-%m-%d')}")
    print(f"Target Date:  {target_date.strftime('%Y-%m-%d')}")

    if current_price:
        print(f"\nCurrent Price: ${current_price:,.2f}")
        print(f"Predicted Price: ${predicted_price:,.2f}")
        print(f"Price Range: ${lower_price:,.2f} to ${upper_price:,.2f}")

    print(f"\nPredicted 7-Day Change: {prediction:.2f}%")
    print(f"Range: {lower_bound:.2f}% to {upper_bound:.2f}%")
    print(f"\nSIGNAL: {signal}")

    # Show trading advice
    print("\nTrading Advice:", end=" ")
    if signal == "BUY":
        print("Strong buying opportunity.")
        if current_price:
            print(f"Stop Loss: ${stop_loss:,.2f} (-5.00%)")
            print(f"Take Profit: ${take_profit:,.2f} (+{(take_profit/current_price-1)*100:.2f}%)")
    elif signal == "WEAK BUY":
        print("Mild bullish signal.")
    elif signal == "SELL":
        print("Strong selling opportunity.")
        if current_price:
            print(f"Stop Loss: ${stop_loss:,.2f} (+5.00%)")
            print(f"Take Profit: ${take_profit:,.2f} ({(take_profit/current_price-1)*100:.2f}%)")
    elif signal == "WEAK SELL":
        print("Mild bearish signal.")
    else:
        print("Hold current positions or stay out of the market.")

    print(f"{'='*60}")

    # Visualize prediction
    plt.figure(figsize=(10, 6))
    if current_price:
        dates = [current_date, target_date]
        plt.scatter([current_date], [current_price], color='blue', s=100, label='Current Price')
        plt.scatter([target_date], [predicted_price], color='red', s=100, label='Predicted Price')
        plt.plot(dates, [current_price, predicted_price], 'r--')

        # Show confidence interval
        plt.fill_between(
            [target_date-pd.Timedelta(days=0.5), target_date+pd.Timedelta(days=0.5)],
            [lower_price, lower_price],
            [upper_price, upper_price],
            color='red', alpha=0.2
        )
        plt.title(f'Bitcoin 7-Day Price Prediction: {prediction:.2f}%')
        plt.ylabel('Price ($)')
    else:
        # Just show percentage change
        plt.bar(['Predicted Change'], [prediction], color='red')
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        plt.title(f'Predicted 7-Day Change: {prediction:.2f}%')
        plt.ylabel('Percent Change (%)')

    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    return {
        'current_date': current_date,
        'target_date': target_date,
        'prediction_percent': prediction,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'signal': signal,
        'current_price': current_price if current_price else None,
        'predicted_price': predicted_price if current_price else None
    }

In [ ]:
# Predict % of bitcoin price change in 7 days (assuming the dataset is up-to-date)
prediction = predict_future_bitcoin_change(
    model=regressor,
    current_data=final_df,
    confidence_level=0.95
)